In [1]:
from DocIndex import DocIndex, create_document_index, CallGpt
from typing import List, Dict
import glob
from langchain.prompts import PromptTemplate
import os
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4")

In [1]:

indexed_docs: Dict[str, DocIndex] = {}
bm25_corpus: List[List[str]] = []
doc_id_to_bm25_index: Dict[str, int] = {}
def load_documents(folder):
    global indexed_docs, bm25_corpus, doc_id_to_bm25_index
    for filepath in glob.glob(os.path.join(folder, '*.index')):
        filename = os.path.basename(filepath)
        doc_index = DocIndex.load_local(folder, filename)
        indexed_docs[doc_index.doc_id] = doc_index

streaming_more_details = PromptTemplate(
            input_variables=["query", "answer", "fragment", "summary", "full_summary", "questions_answers"],
            template="""
Our user had previously asked us a query to answer from a large document. 
We had answered the question and now the user has asked a for more details on the same query and answer.
You will help further details from context (text chunks of larger document) you are provided. 
The provided context is part/fragment of a larger main document.

The original question or request is given below:

{query}


You are also given the summary of the larger main document below:

{full_summary}

You are given few text chunks from within the document to answer this question as below:

{fragment}

Next, You are given few question and answer pairs from the document below:

{questions_answers}

You are also given summarized text chunks of certain parts of document below:

{summary}


Answer elaborately providing as much detail as possible. Remember the user is seeking further information over what is already provided in your previous answer. Continue expanding on your previous answer with more details and elaboration. Be detail oriented.

Question: {query}
Answer till now: {answer}

""",
        )
        

In [8]:
load_documents("storage")

In [9]:
indexed_docs

indexed_docs['1311701502'].streaming_more_details

PromptTemplate(input_variables=['query', 'answer', 'fragment', 'summary', 'full_summary', 'questions_answers'], output_parser=None, partial_variables={}, template='\nOur user had previously asked us a query to answer from a large document. \nWe had answered the question and now the user has asked a for more details on the same query and answer.\nYou will help further details from context (text chunks of larger document) you are provided. \nThe provided context is part/fragment of a larger main document.\n\nThe original question or request is given below:\n\n{query}\n\n\nYou are also given the summary of the larger main document below:\n\n{full_summary}\n\nYou are given few text chunks from within the document to answer this question as below:\n\n{fragment}\n\nNext, You are given few question and answer pairs from the document below:\n\n{questions_answers}\n\nYou are also given summarized text chunks of certain parts of document below:\n\n{summary}\n\n\nAnswer elaborately providing as m

In [6]:
setattr(indexed_docs['1311701502'], "streaming_more_details", streaming_more_details)


In [7]:
indexed_docs['1311701502'].save_local("storage")

In [ ]:
# Quick Index -> Show a running summary being written for whole paper (use the streaming_more_details style)
    # Make it in same view but keep QnA button disabled first.
# Similar Papers and web search

# Streaming -> methodology and other stuff as well. Just like summary

# Use the new title and summary in your queries

# Check duplicates for new doc addition

# Enable search

# Web search , references and citations

# Collections

# Long Aspect read



# OpenAI api fault tolerance

# Clean-up

# Resilient Later indexing

# Multi-account and Locks on API for concurrency

# Crowd sourced content and qna is important, People should be able to answer as well.

# Upvote downvote for an answer, Comments for an answer.

# Search, Reader and Cites + Refs Ability.

# User id and user level conversations or forums for reading -> Peer reading

# Use twitter to get relevant conversations for a paper/work to seed it.

# Voice Inputs

# Second reading (First Reading anyway has to be full?)

# what does the first part or first chunk or "x" part of document say



https://replit.com/
https://github.com/mckaywrigley/chatbot-ui


In [24]:
from langchain.document_loaders import PyPDFLoader
pdf_url = "https://arxiv.org/pdf/2304.05187.pdf"
loader = PyPDFLoader(pdf_url)
pages = loader.load_and_split()

In [39]:
from langchain.document_loaders import PDFMinerLoader
loader = PDFMinerLoader(pdf_url)
data = loader.load()
len(enc.encode(" ".join([p.page_content for p in data])))

20931

In [25]:
from langchain.document_loaders import OnlinePDFLoader
loader = OnlinePDFLoader(pdf_url)
data = loader.load()
print(data)


[Document(page_content='3 2 0 2\n\nr p A 1 1\n\n]\n\nG L . s c [\n\n1 v 7 8 1 5 0 . 4 0 3 2 : v i X r a\n\nAutomatic Gradient Descent: Deep Learning without Hyperparameters\n\nJeremy Bernstein‹ MIT\n\nChris Mingard‹ U. Oxford\n\nKevin Huang U. Washington\n\nNavid Azizan MIT\n\nYisong Yue Caltech\n\n‹ denotes equal contribution.\n\nAbstract\n\nThe architecture of a deep neural network is deﬁned explicitly in terms of the number of layers, the width of each layer and the general network topology. Existing optimisation frameworks neglect this information in favour of implicit architectural information (e.g. second-order methods) or architecture-agnostic distance functions (e.g. mirror descent). Meanwhile, the most popular optimiser in practice—Adam—is based on heuristics. This paper builds a new framework for deriving optimisation algorithms that explicitly leverage neural architecture. The theory extends mirror descent to non-convex composite objective functions: the idea is to transform

In [2]:
def create_semantic_blocks_based_on_font_size(pdf_url):
    from langchain.document_loaders import PDFMinerPDFasHTMLLoader
    loader = PDFMinerPDFasHTMLLoader(pdf_url)
    data = loader.load()[0]

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(data.page_content,'html.parser')
    content = soup.find_all('div')
    import re
    cur_fs = None
    cur_text = ''
    snippets = []   # first collect all snippets that have the same font size
    for c in content:
        sp = c.find('span')
        if not sp:
            continue
        st = sp.get('style')
        if not st:
            continue
        fs = re.findall('font-size:(\d+)px',st)
        if not fs:
            continue
        fs = int(fs[0])
        if not cur_fs:
            cur_fs = fs
        if fs == cur_fs:
            cur_text += c.text
        else:
            snippets.append((cur_text,cur_fs))
            cur_fs = fs
            cur_text = c.text
    snippets.append((cur_text,cur_fs))
    # Note: The above logic is very straightforward. One can also add more strategies such as removing duplicate snippets (as
    # headers/footers in a PDF appear on multiple pages so if we find duplicatess safe to assume that it is redundant info)
    from langchain.docstore.document import Document
    cur_idx = -1
    semantic_snippets = []
    # Assumption: headings have higher font size than their respective content
    for s in snippets:
        # if current snippet's font size > previous section's heading => it is a new heading
        if not semantic_snippets or s[1] > semantic_snippets[cur_idx].metadata['heading_font']:
            metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
            metadata.update(data.metadata)
            semantic_snippets.append(Document(page_content='',metadata=metadata))
            cur_idx += 1
            continue

        # if current snippet's font size <= previous section's content => content belongs to the same section (one can also create
        # a tree like structure for sub sections if needed but that may require some more thinking and may be data specific)
        if not semantic_snippets[cur_idx].metadata['content_font'] or s[1] <= semantic_snippets[cur_idx].metadata['content_font']:
            semantic_snippets[cur_idx].page_content += s[0]
            semantic_snippets[cur_idx].metadata['content_font'] = max(s[1], semantic_snippets[cur_idx].metadata['content_font'])
            continue

        # if current snippet's font size > previous section's content but less tha previous section's heading than also make a new 
        # section (e.g. title of a pdf will have the highest font size but we don't want it to subsume all sections)
        metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
        metadata.update(data.metadata)
        semantic_snippets.append(Document(page_content='',metadata=metadata))
        cur_idx += 1
    semantic_snippets = [s.page_content.strip() for s in semantic_snippets]
    semantic_snippets = [s for s in semantic_snippets if len(s)>0]
    return semantic_snippets
    
    

In [3]:
pdf_url = "https://arxiv.org/pdf/2304.05187.pdf"
semantic_snippets = create_semantic_blocks_based_on_font_size(pdf_url)
print(len(semantic_snippets))
# print(semantic_snippets[4])
len(enc.encode(" ".join(semantic_snippets)))


12


20402

In [12]:
# print([i for i, c in enumerate(chunks) if "Algorithm" in c])

# display_markdown(chunks[10], raw=True)

semantic_snippets[7]

'In this section, we will focus our eﬀorts on deriving an optimiser for deep fully-connected networks trained\nwith square loss. The derivation for cross-entropy loss is analogous. Proofs are relegated to Appendix A.\nDeﬁnition 10 (Fully-connected network) A fully-connected network (FCN) f of depth L maps an input x P Rd0\nto an output f px; wq P RdL via L matrix multiplications interspersed by non-linearity relupzq :“ maxp0, zq:\nf px; wq :“ WL ˝ prelu ˝ WL´1q ˝ prelu ˝ WL´2q ˝ ¨ ¨ ¨ ˝ prelu ˝ W1xq.\nIn this expression, w denotes the tuple of matrices w “ pW1, ..., WLq with kth matrix Wk in Rdkˆdk´1. In\nwhat follows, we will ﬁnd the following dimensional scaling to be particularly convenient:\nPrescription 1 (Dimensional scaling) For η ą 0, the data px, yq, weights Wk and updates ∆Wk should obey:\n}x}2 “\n}Wk}˚ “\n}∆Wk}˚ “\n}y}2 “\na\nd0;\na\na\na\ndk{dk´1\ndk{dk´1 ¨ η\nL\ndL.\nfor all k “ 1, ..., L;\nfor all k “ 1, ..., L;\n(input scaling)\n(weight scaling)\n(update scaling)\n(targe

In [13]:
from DocIndex import PDFReaderTool, ChunkText, call_ai21

In [40]:


doc_text = PDFReaderTool()("https://arxiv.org/pdf/2304.05187.pdf")
chunks = ChunkText(doc_text, 1024, 128)
len(chunks)

Status: loaded, waiting for processing to complete
Status: split, waiting for processing to complete
Status: split, waiting for processing to complete


29

In [18]:
prompt = f"""
Given below text is small part/fragment text of a larger document. Based on the below fragment of document, write a set of important key phrases or sentences in elaborate bullet points that can be derived from the fragment. We will use these key phrases or sentences to summarize the document later.

"{semantic_snippets[7][:9000]}"

Note: This is a raw unformatted document which may contain irrelevant text/syntax or other noise. Ignore what you consider noise or irrelevant.
Based on the fragment of document, write a set of important key phrases (keywords) or sentences in elaborate bullet points that can be derived from the fragment.
Don't give out small bullet points.

Key phrases or keywords or detailed outline in markdown bullet points:

"""

In [19]:
import cohere  
co = cohere.Client("0GFuzMkYpofxfwIAxSf142NPkcICrHuVuPN4kaca")

response = co.generate(  
    model='command-nightly',  
    prompt = prompt,  
    max_tokens=512,  
    temperature=0.6)
intro_paragraph = response.generations[0].text
print(intro_paragraph)

- We present a novel derivation of automatic gradient descent for deep fully-connected networks trained
with square loss.
- The derivation is based on majorisation-minimisation principles.
- We show that the resulting algorithm converges to a critical point where the gradient vanishes.
- We also present a Polyak-Łojasiewicz inequality tailored to the operator structure of deep networks.


In [20]:
print(call_ai21(prompt, temperature=0.7))

Calling POST https://api.ai21.com/studio/v1/j2-jumbo-instruct/complete failed with Exception: HTTPSConnectionPool(host='api.ai21.com', port=443): Read timed out. (read timeout=30)


1. Derivation of an optimiser for fully-connected networks trained with square loss
1. Definition 10 (Fully-connected network)
1. Prescription 1 (Dimensional scaling)
1. Lemma 3 (Output bound)
1. Lemma 4 (Deep relative trust)
1. Lemma 5 (Exponential majorisation)
1. Algorithm 1 (Automatic gradient descent)
1. Lemma 6 (Bounded objective)
1. Lemma 7 (Bounded gradient)
1. Theorem 2 (Automatic gradient descent)
1. Lemma 8 (Convergence rate to critical point)
1. Assumption 3 (Deep Polyak-Łojasiewicz inequality)


In [5]:
from IPython.display import display_markdown
display_markdown(doc_text, raw=True)

\title{
RWKV: Reinventing RNNs for the Transformer Era
}

\author{
Bo Peng ({ }^{1 *}) Eric Alcaide ({ }^{2,3,4 *}) Quentin Anthony ({ }^{2,5 *}) \\ Alon Albalak ({ }^{2,6}) Samuel Arcadinho (^{2,7}) Huanqi Cao (^{8}) Xin Cheng (^{9}) Michael Chung (^{10}) \\ Matteo Grella ({ }^{11}) Kranthi Kiran GV ({ }^{12}) Xuzheng (He ^{2}) Haowen Hou ({ }^{13}) Przemysaw Kazienko ({ }^{14}) \\ Jan Kocoń ({ }^{14}) Jiaming Kong ({ }^{15}) Bartłomiej Koptyra ({ }^{14}) Hayden Lau (^{2}) Krishna Sri Ipsit Mantri ({ }^{16}) \\ Ferdinand Mom ({ }^{17,18}) Atsushi Saito ({ }^{2,19}) Xiangru Tang ({ }^{20}) Bolun Wang ({ }^{27}) Johan S. Wind ({ }^{21}) Stanisław Woźniak (^{14}) \\ Ruichong Zhang ({ }^{8}) Zhenyuan Zhang ({ }^{2}) Qihang Zhao ({ }^{22,23}) Peng Zhou ({ }^{27}) Jian Zhu ({ }^{24}) Rui-Jie Zhu ({ }^{25,26}) \\ ({ }^{1}) RWKV Foundation ({ }^{2}) EleutherAI ({ }^{3}) University of Barcelona ({ }^{4}) Charm Therapeutics ({ }^{5}) Ohio State University \\ ({ }^{6}) University of California, Santa Barbara ({ }^{7}) Zendesk ({ }^{8}) Tsinghua University ({ }^{9}) Peking University \\ ({ }^{10}) Storyteller.io ({ }^{11}) Crisis (24{ }^{12}) New York University ({ }^{13}) National University of Singapore \\ ({ }^{14}) Wroclaw University of Science and Technology ({ }^{15}) Databaker Technology Co. Ltd ({ }^{16}) Purdue University \\ ({ }^{17}) Criteo AI Lab ({ }^{18}) Epita ({ }^{19}) Nextremer Co. Ltd. ({ }^{20}) Yale University ({ }^{21}) University of Oslo \\ ({ }^{22}) University of Science and Technology of China ({ }^{23}) Kuaishou Technology Co. Ltd \\ ({ }^{24}) University of British Columbia ({ }^{25}) University of California, Santa Cruz \\ ({ }^{26}) University of Electronic Science and Technology of China ({ }^{27}) RuoxinTech
}

\begin{abstract}
Transformers have revolutionized almost all natural language processing (NLP) tasks but suffer from memory and computational complexity that scales quadratically with sequence length. In contrast, recurrent neural networks (RNNs) exhibit linear scaling in memory and computational requirements but struggle to match the same performance as Transformers due to limitations in parallelization and scalability. We propose a novel model architecture, Receptance Weighted Key Value (RWKV), that combines the efficient parallelizable training of Transformers with the efficient inference of RNNs. Our approach leverages a linear attention mechanism and allows us to formulate the model as either a Transformer or an (RNN), which parallelizes computations during training and maintains constant computational and memory complexity during inference, leading to the first non-transformer architecture to be scaled to tens of billions of parameters. Our experiments reveal that RWKV performs on par with similarly sized Transformers, suggesting that future work can leverage this architecture to create more efficient models. This work presents a significant step towards reconciling the trade-offs between computational efficiency and model performance in sequence processing tasks. ({ }^{1})
\end{abstract}

\section{Introduction}

Deep learning techniques have made significant strides in artificial intelligence, playing a pivotal

* Equal first authorship. Others listed alphabetically.

({ }^{1}) Code at: https://github.com/BlinkDL/RWKV-LM role in various scientific and industrial applications. These applications often involve complex sequential data processing tasks that include natural language understanding, conversational AI, time-series analysis, and even indirect modalities that can be reframed as sequences, such as images and graphs (Brown et al., 2020; Ismail Fawaz et al., 2019; Wu et al., 2020; Albalak et al., 2022). Predominant among these techniques are RNNs, convolutional neural networks (CNNs), and the Transformer models (Vaswani et al., 2017).

Each of these has distinct drawbacks that restrict their efficiency in certain scenarios. RNNs suffer from the vanishing gradient problem, making them difficult to train for long sequences. Additionally, they cannot be parallelized in the time dimension during training, which restricts their scalability (Hochreiter, 1998; Le and Zuidema, 2016). CNNs, on the other hand, are only adept at capturing local patterns, which limits their capacity to deal with long-range dependencies, crucial to many sequence processing tasks (Bai et al., 2018).

Transformer models emerged as a powerful alternative due to their ability to handle both local and long-range dependencies and their capability for parallelized training (Tay et al., 2022). Recent models such as GPT-3 (Brown et al., 2020), ChatGPT (OpenAI, 2022; Kocoń et al., 2023), GPT-4 (OpenAI, 2023), LLaMA (Touvron et al., 2023), and Chinchilla (Hoffmann et al., 2022) exemplify the capability of this architecture, pushing the frontiers of what's possible in NLP. Despite these significant advancements, the self-attention mechanism inherent to Transformers poses unique challenges, 

\begin{tabular}{lcc}
\hline Model & Time & Space \\
\hline Transformer & (O\left(T^{2} d\right)) & (O\left(T^{2}+T d\right)) \\
Reformer & (O(T \log T d)) & (O(T \log T+T d)) \\
Linear Transformers & (O\left(T d^{2}\right)) & (O\left(T d+d^{2}\right)) \\
Performer & (O\left(T d^{2} \log d\right)) & (O\left(T d \log d+d^{2} \log d\right)) \\
AFT-full & (O\left(T^{2} d\right)) & (O(T d)) \\
MEGA & (O(c T d)) & (O(c T d)) \\
\hline RWKV (ours) & (O( T d )) & (O( d )) \\
\hline
\end{tabular}

Table 1: Complexity comparison with different Transformers: Reformer (Kitaev et al., 2020), Linear Transformer (Katharopoulos et al., 2020), Performer (Choromanski et al., 2020), AFT (Zhai et al., 2021), MEGA (Ma et al., 2023). Here (T) denotes the sequence length, (d) the feature dimension, and (c) is MEGA's chunk size of quadratic attention.

primarily due to its quadratic complexity. This complexity renders the architecture computationally expensive and memory-intensive for tasks involving long input sequences or in resource-constrained situations. These limitations have spurred a wealth of research aiming to improve the scaling properties of Transformers, often at the expense of some of the properties that make it so effective (Wang et al., 2020; Zaheer et al., 2020; Dao et al., 2022a).

To tackle these challenges, we introduce the Receptance Weighted Key Value (RWKV) model, a novel architecture that effectively combines the strengths of RNNs and Transformers while circumventing key drawbacks. RWKV is carefully designed to alleviate the memory bottleneck and quadratic scaling associated with Transformers (Katharopoulos et al., 2020) with a more efficient linear scaling, while still preserving the rich, expressive properties that make the Transformer a dominant architecture in the field.

One of the defining characteristics of RWKV is its ability to offer parallelized training and robust scalability, similar to Transformers. Moreover, we have reformulated the attention mechanism in RWKV to introduce a variant of linear attention, eschewing the traditional dot-product token interaction in favor of more effective channeldirected attention. This approach contrasts significantly with the traditional Transformer architecture, where specific token interactions predominantly drive attention. The implementation of linear attention in RWKV is carried out without approximation, which offers a considerable improvement in efficiency and enhances the scalability, see Table 1.

The overarching motivation behind developing RWKV is to bridge the gap between computational efficiency and expressive capacity in neural net- work architectures. It offers a promising and viable solution for handling tasks involving large-scale models with billions of parameters, exhibiting competitive performance at a fraction of the computational cost. Our experimental results suggest that RWKV could be a valuable tool for addressing the ongoing challenges in scaling and deploying AI models across various domains, particularly those involving sequential data processing. Thus, RWKV paves the way for the next generation of more sustainable and computationally efficient AI models for sequence processing tasks.

Our contributions in this paper are as follows:

- We introduce the RWKV network architecture, which combines the advantages of RNNs and Transformers while mitigating their known limitations.

- We propose a new attention mechanism reformulation that results in linear attention, eschewing the quadratic complexity associated with standard Transformer models.

- We conduct a comprehensive series of experiments on benchmark datasets to showcase the performance, efficiency and scaling of RWKV in managing tasks involving large-scale models and long-range dependencies.

- We release pretrained model ranging in size from 169 million to 14 billion parameters trained on the Pile (Gao et al., 2020). ({ }^{2})

\section{Related Work}

Recently, a number of techniques have been proposed to address the limitations of transformers.

Optimizing Attention Mechanism Many transformer variants ("x-formers") have been introduced to reduce the complexity of transformers (Tay et al., 2022), including sparse attention (Beltagy et al., 2020; Kitaev et al., 2020; Guo et al., 2022), approximating the full attention matrix (Wang et al., 2020; Ma et al., 2021; Choromanski et al., 2020), combining chunked attention with gating (Ma et al., 2023) and other efficient methods (Katharopoulos et al., 2020; Jaegle et al., 2021).

Some recent works like FlashAttention (Dao et al., 2022a) and others (Rabe and Staats, 2022; Jang et al., 2019) share similarities with RWKV's chunked computation scheme. Despite being memory-efficient, their time complexity remains quadratic or contains chunk size as a hidden factor. In contrast, RWKV achieves better space and

({ }^{2} https : / /) huggingface.co/RWKV time complexity during inference by formulating a linear attention as an (RNN).

Attention Free Models Another line of research replaces the attention mechanism with other modules to scale to long sequences. MLP-Mixer and others (Tolstikhin et al., 2021; Liu et al., 2021) proposed the replacement of attention by MultiLayer Perceptrons (MLPs) in computer vision tasks. The Attention Free Transformer (AFT) (Zhai et al., 2021) replaces dot-product self-attention with a computationally efficient alternative which can be seen as a multi-head attention where each feature dimension corresponds to a head. Inspired by AFT, RWKV takes a similar approach but modifies the interaction weights for simplicity such that it can be transformed into an RNN. In parallel, RNNstyle (Hochreiter and Schmidhuber, 1997; Chung et al., 2014) recursive components have also been modified to increase context length, such as the Recurrent Memory Transformer (Bulatov et al., 2022, 2023) and Linear Recurrent Units (Orvieto et al., 2023). State space models (SSM) like S4 (Gu et al., 2022) and its variants (Dao et al., 2022b; Poli et al., 2023) are also proposed.

Notably, Quasi-Recurrent neural network (QRNN) (Bradbury et al., 2017) uses both convolutional layers and recurrent pooling functions across timesteps and channels. While QRNN utilizes convolutional filters with fixed sizes, RWKV employs a time-mixing module as an attention mechanism with time-decaying factors. Different from the element-wise pooling in QRNN, RWKV includes a parametrized channel-mixing module (see the green blocks in Fig.1c) that is parallelizable.

\section{Background}

Here we briefly review the fundamentals of RNNs and Transformers.

\subsection{Recurrent Neural Networks (RNNs)}

Popular RNN architectures such as LSTM (Hochreiter and Schmidhuber, 1997) and GRU (Chung et al., 2014) are characterized by the following formulation (shown for LSTM, others can be reasoned similarly):

\[
\begin{aligned}
f_{t} & =\sigma_{g}\left(W_{f} x_{t}+U_{f} h_{t-1}+b_{f}\right), \\
i_{t} & =\sigma_{g}\left(W_{i} x_{t}+U_{i} h_{t-1}+b_{i}\right), \\
o_{t} & =\sigma_{g}\left(W_{o} x_{t}+U_{o} h_{t-1}+b_{o}\right), \\
\tilde{c}_{t} & =\sigma_{c}\left(W_{c} x_{t}+U_{c} h_{t-1}+b_{c}\right), \\
c_{t} & =f_{t} \odot c_{t-1}+i_{t} \odot \tilde{c}_{t}, \\
h_{t} & =o_{t} \odot \sigma_{h}\left(c_{t}\right) .
\end{aligned}
\]

The data flow of RNNs is shown in Fig. 1a. Although RNNs can be factored into two linear blocks ( (W) and (U) ) and an RNN-specific block (1)-(6), as noted by Bradbury et al. (2017), the data dependency relying on previous time steps prohibits parallelizing these typical RNNs.

\subsection{Transformers and AFT}

Introduced by Vaswani et al. (2017), Transformers are a class of neural networks that have become the dominant architecture for several NLP tasks. Instead of operating on sequences step-by-step like RNNs, Transformers rely on attention mechanisms to capture relationships between all input and all output tokens:

\[
\operatorname{Attn}(Q, K, V)=\operatorname{softmax}\left(Q K^{\top}\right) V
\]

where the multi-headness and scaling factor (\frac{1}{\sqrt{d_{k}}}) is omitted for convenience. The core (Q K^{\top}) multiplication is an ensemble of pairwise attention scores between each token in a sequence, which can be decomposed as vector operations:

\[
\operatorname{Attn}(Q, K, V)_{t}=\frac{\sum_{i=1}^{T} e^{q_{t}^{\top} k_{i}} v_{i}}{\sum_{i=1}^{T} e^{q_{t}^{\top} k_{i}}}
\]

In AFT (Zhai et al., 2021), this is alternately formulated as

\[
\operatorname{Attn}^{+}(W, K, V)_{t}=\frac{\sum_{i=1}^{t} e^{w_{t, i}+k_{i}} v_{i}}{\sum_{i=1}^{t} e^{w_{t, i}+k_{i}}}
\]

where (\left\{w_{t, i}\right\} \in R^{T \times T}) is the learned pair-wise position biases, and each (w_{t, i}) is a scalar.

Inspired by AFT, we let each (w_{t, i}) in RWKV be a channel-wise time decay vector multiplied by the relative position, traced backwards from current time as it decays:

\[
w_{t, i}=-(t-i) w
\]

where (w \in\left(R_{\geq 0}\right)^{d}), with (d) the number of channels. We require (w) to be non-negative to ensure that (e^{w_{t, i}} \leq 1) and the per-channel weights decay backwards in time. 


(a) RNN


(b) QuasiRNN (Bradbury et al., 2017)


(c) (RWKV)

Figure 1: Computation structure of the RWKV in comparison to QRNN and RNN (Vanilla, LSTM, GRU, etc) architectures. Color codes: orange indicates time-mixing, convolutions or matrix multiplications, and the continuous block indicates that these computations can proceed simultaneously; blue signifies parameterless functions that operate concurrently along the channel or feature dimension (element-wise). Green indicates channel-mixing.

\section{The Receptance Weighted Key Value (RWKV) Model}

The RWKV architecture derives its name from the four primary model elements used in the timemixing and channel-mixing blocks:

- (R) : Receptance vector acting as the acceptance of past information.

- (W) : Weight is the positional weight decay vector. A trainable model parameter.

- (K) : Key is a vector analogous to (K) in traditional attention.

- (V) : Value is a vector analogous to (V) in traditional attention.

Interactions between the main elements for every timestep are multiplicative, as illustrated in Fig. 2

\subsection{High-Level Summary}

The RWKV architecture is comprised of a series of stacked residual blocks, each formed by a timemixing and a channel-mixing sub-blocks with recurrent structures.

The recurrence is formulated both as a linear interpolation between the current input and the input at the previous time step (a technique we refer to as time-shift mixing or token shift, indicated by the diagonal lines in Fig. 3), which can be adjusted independently for every linear projection of the input embedding (e.g., (R, K, V) in time-mixing, and (R), (K) in channel-mixing), and as the time-dependent update of the (W K V) which is formalized in equation 14. The (W K V) computation is similar to AFT (Zhai et al., 2021), but (W) is now a channel-wise vector multiplied by relative position rather than a pairwise matrix in AFT. We also introduce a vector (U) for separately attending to the current token in order to compensate for potential degeneration of (W) (see Appendix G for more details).


Figure 2: RWKV block elements (left) and RWKV residual block with a final head for language modeling (right) architectures.


Figure 3: RWKV architecture for language modelling. The time-mixing block is given by:

\[
\begin{aligned}
r_{t} & =W_{r} \cdot\left(\mu_{r} x_{t}+\left(1-\mu_{r}\right) x_{t-1}\right) \\
k_{t} & =W_{k} \cdot\left(\mu_{k} x_{t}+\left(1-\mu_{k}\right) x_{t-1}\right) \\
v_{t} & =W_{v} \cdot\left(\mu_{v} x_{t}+\left(1-\mu_{v}\right) x_{t-1}\right) \\
w k v_{t} & =\frac{\sum_{i=1}^{t-1} e^{-(t-1-i) w+k_{i}} v_{i}+e^{u+k_{t}} v_{t}}{\sum_{i=1}^{t-1} e^{-(t-1-i) w+k_{i}}+e^{u+k_{t}}} \\
o_{t} & =W_{o} \cdot\left(\sigma\left(r_{t}\right) \odot w k v_{t}\right)
\end{aligned}
\]

where the (W K V) computation, (w k v_{t}), plays the role of (\operatorname{Attn}(Q, K, V)) in Transformers without incurring a quadratic cost as interactions are between scalars. Intuitively, as time (t) increases, the vector (o_{t}) is dependent on a long history, represented by the summation of an increasing number of terms. For the target position (t), RWKV performs a weighted summation in the positional interval of ([1, t]), and then multiplies with the receptance (\sigma(r)). Therefore, interactions are multiplicative inside a given timestep and summed over different timesteps.

Further, the channel-mixing block is given by:

\[
\begin{aligned}
r_{t} & =W_{r} \cdot\left(\mu_{r} x_{t}+\left(1-\mu_{r}\right) x_{t-1}\right), \\
k_{t} & =W_{k} \cdot\left(\mu_{k} x_{t}+\left(1-\mu_{k}\right) x_{t-1}\right), \\
o_{t} & =\sigma\left(r_{t}\right) \odot\left(W_{v} \cdot \max \left(k_{t}, 0\right)^{2}\right),
\end{aligned}
\]

where we adopt squared ReLU activation (So et al., 2021). Note that in both time-mixing and channelmixing, by taking the sigmoid of the receptance, we're intuitively using it as a "forget gate" to eliminate unnecessary historical information.

\subsection{Transformer-like Parallelization}

RWKV can be efficiently parallelized in what we call a time-parallel mode, reminiscent of Transformers. The time complexity of processing a batch of sequences in a single layer is (O\left(B T d^{2}\right)), which mainly consists of matrix multiplications (W_{\square}, \square \in\{r, k, v, o\}) (assuming (B) sequences, (T) maximum tokens and (d) channels). Meanwhile, updating attention scores (w k v_{t}) requires a serial scan (see Appendix B for more detail) and has complexity (O(B T d)).

The matrix multiplications can be parallelized akin to (W^{\square}, \square \in\{Q, K, V, O\}) in typical Transformers. The element-wise (W K V) computation is time-dependent, but can be readily parallelized along the other two dimensions (Lei et al., 2018) ({ }^{3}).

({ }^{3}) If the sequence is very long, more sophisticated methods such as Martin and Cundy (2017) that parallelize over sequence length could be used. Additionally, token shift is implemented as a simple offset in the temporal dimension at each block using PyTorch (Paszke et al., 2019) library as (nn). (\operatorname{ZeroPad} 2 d ((\theta, \theta, 1,-1))).

\subsection{RNN-like Sequential Decoding}

It is common in recurrent networks to use output at state (t) as input at state (t+1). This is especially evident in the autoregressive decoding inference of a language model, requiring each token to be computed before fed into the next step, making it possible for RWKV to take advantage of its RNNlike structure, referred to as time-sequential mode. In such circumstances, RWKV can be conveniently formulated recursively for decoding during inference, as shown in Appendix B, which leverages the advantage that each output token is dependent only on the latest state, which is of constant size, irrespective of the sequence length.

It then behaves as an RNN decoder, yielding constant speed and memory footprint with respect to the sequence length, enabling the processing of longer sequences more efficiently. In contrast, selfattention typically requires a (KV) cache growing linearly with respect to the sequence length, resulting in degraded efficiency and increasing memory footprint and time as the sequence grows longer.

\subsection{Software Implementation}

RWKV is originally implemented using the Pytorch Deep Learning Library (Paszke et al., 2019) and a custom CUDA kernel for the (W K V) computation explained in 4.7. Although RWKV is a general recurrent network, its current implementation focuses in the task of language modeling (RWKV-LM). The model architecture is comprised of an embedding layer, for which we follow the setup described in Section 4.7 and several identical residual blocks applied sequentially as seen in Fig. 2 and 3 following the principles outlined in Section 4.6. After the last block, a simple output projection head composed by a LayerNorm (Ba et al., 2016) and a linear projection is used to obtain the logits to be used in the next-token prediction task and calculate the cross entropy loss during training. Both the embeddings generated after the last residual block and the logits could also be used later for downstream NLP tasks. Training is performed in time-parallel mode (Section 4.2) while autoregressive inference and a potential chat interface ({ }^{4})

({ }^{4} https : / /) github.com/BlinkDL/ChatRWKV leverage the time-sequential mode (Section 4.3).

\subsection{Gradient Stability and Layer Stacking}

The RWKV architecture has been designed as a fusion of both Transformers and RNNs, offering the advantage of stable gradients and deeper architectures of Transformers compared to traditional RNNs while being efficient in inference.

Previous work has sought to tackle the problem of gradient stability in RNNs with a variety of techniques including using non-saturated activation functions (Chandar et al., 2019), gating mechanism (Gu et al., 2019), gradient clipping (Pascanu et al., 2012), and adding constraints (Kanai et al., 2017; Miller and Hardt, 2018). While these techniques have seen little success, RWKV avoids the problem inherently by utilizing softmax in conjunction with RNN-style updates.

The RWKV model features a single-step process for updating attention-like scores, which includes a time-dependent softmax operation that helps numerical stability and guards against vanishing gradients (for rigorous proof, see Appendix F). Intuitively, this operation ensures the gradient is propagated along the most relevant path. Layer normalization (Ba et al., 2016) is another key aspect of the architecture which enhances the training dynamics of deep neural networks by stabilizing gradients, addressing both vanishing and exploding gradient issues.

These design elements not only contribute to the RWKV architecture's stability and learning capabilities but enable the stacking of multiple layers in a manner that surpasses the capabilities of any existing RNN. In doing so, the model is able to capture more complex patterns across various levels of abstraction (see also Appendix G).

\subsection{Harnessing Temporal Structure for Sequential Data Processing}

RWKV captures and propagates sequential information through the combination of three mechanisms: recurrence, time decay and token shift.

The recurrence in the time-mixing block of RWKV is the basis for the model's capacity to capture intricate relationships between sequence elements and to propagate locality information through time.

The time decay mechanism (\left(e^{-w}\right.) and (e^{u}) in equation 14), maintains sensitivity to the positional relationship between sequence elements. By gradually diminishing the influence of past information over time, the model preserves a sense of temporal locality and progression, which is essential for sequential processing. This treatment of positional information in sequential data exhibits similarities to the Attention with Linear Biases (ALiBi) model (Press et al., 2022), where the linear biases facilitate input length extrapolation. In this context, the RWKV architecture can be perceived as a trainable version of (ALiBi), seamlessly incorporating positional information without the necessity for explicit encoding. It can also be seen as an extension of the gated convolution introduced in Zhai et al. (2021) to the full sequence length until a given step.

The token shift or time-shift mixing, or (diagonal arrows in Figure 3), also contributes to the model's adaptation to sequential data. By linearly interpolating between the current input and the previous time step input, the model naturally aggregates and gates information in the input channels. The overall structure of time-shift mixing bears resemblance to the causal convolution with no dilations in WaveNet (van den Oord et al., 2016), which is a classical architecture used for forecasting time series data.

\subsection{Additional Optimizations}

Custom Kernels To address inefficiencies in the (W K V) computation due to the sequential nature of the task when using standard deep learning frameworks, we implement a custom CUDA kernel so as to launch a single compute kernel in training accelerators. All other parts of the model are matrix multiplications and point-wise operations that can already be efficiently parallelized.

FFN with R gate Prior research (Tolstikhin et al., 2021; Liu et al., 2021; Yu et al., 2022) suggests that self-attention may not be as essential in Transformer-based vision tasks as previously thought. Although it provided us with some insights, replacing self-attention entirely in natural language tasks could be too drastic. In our study, we partially dismantle the attention mechanism by replacing the fixed (Q K V) formula with (K V) and introducing a new time-decaying factor (W). This approach enables us to incorporate token and channelmixing components akin to MLP-mixer (Tolstikhin et al., 2021) and a gating unit (R) similar to gMLP (Liu et al., 2021), which enhance the performance of our RWKV model.

Small Init Embedding During the initial stage of training a transformer model (Vaswani et al., 2017), we observe that the embedding matrix undergoes slow changes, which pose a challenge for the model to deviate from its initial noisy embedding state. To mitigate this issue, we propose an approach that involves initializing the embedding matrix with small values and subsequently applying an additional LayerNorm operation. By implementing this technique, we accelerate and stabilize the training process, enabling the training of deep architectures with post-LN components. The effectiveness of this approach is demonstrated in Figure 8 , where it is shown to facilitate improved convergence by allowing the model to quickly transition away from the initially small embedding. This is achieved through small changes following a single step, which in turn lead to substantial alterations in directions and subsequently significant changes after the LayerNorm operation.

Custom Initialization Building on principles from previous works (He et al., 2016; Jumper et al., 2021), we initialize parameters to values as similar as possible to an identity mapping while breaking symmetry so there is a clean information path. Most weights are initialized to zero. No biases are used for linear layers. Specific formulas are given in Appendix D. We find the choice of initialization to be significant in convergence speed and quality (see Appendix E).

\section{Evaluations}

In this section, we focus on evaluating to answer the following questions:

- RQ1: Is RWKV competitive against quadratic transformer architectures with equal number of parameters and training tokens?

- RQ2: When increasing the number of parameters, does RWKV remain competitive against quadratic transformer architectures?

- RQ3: Does increasing parameters of RWKV yield better language modeling loss, when RWKV models are trained for context lengths that most open-sourced quadratic transformers cannot efficiently process?

Addressing RQ1 and RQ2, from Fig. 4, we can see that RWKV is very competitive on six benchmarks (Winogrande, PIQA, ARC-C, ARC-E, LAMBADA, and SciQ) against major open source quadratic complexity transformer models: Pythia (Biderman et al., 2023), OPT (Zhang et al., 2022) and BLOOM (Scao et al., 2022). RWKV even outperforms Pythia and GPT-Neo (Black et al., 2022) in four tasks: PIQA, OBQA, ARC-E, and COPA (See details in Appendix H). For RQ3, Fig. 5 shows that increasing context length leads to lower test loss on the Pile, an indication that RWKV can make effective use of long contextual information.

\section{Inference Experiments}

We benchmark inference requirements according to size and family. Specifically, we evaluate text generation speed and memory requirements on a typical compute platforms including CPU (x86) and GPU (NVIDIA A100 80GB). For all our experiments we use float 32 precision. We include all model parameters in parameter count, including both embedding and non-embedding layers. Performance under different quantization setups is left to further work. See Appendix I for more results.


Figure 6: Cumulative time during text generation for different LLMs.

Additionally, we carried out comparative studies on RWKV-4 and ChatGPT / GPT-4, see Appendix J. They revealed that RWKV-4 is very sensitive to prompt engineering. When the prompts were adjusted from the ones used for GPT to more suitable for RWKV, the F1-measure performance increased even from (44.2 %) to (74.8 %).

\section{Future Work}

There are several promising directions for future work on the RWKV architecture:

- Increasing model expressivity with enhanced time-decay formulations and exploring initial model states while maintaining efficiency. 


(a) Winogrande


(d) ARC-Easy


(b) PIQA


(e) LAMBADA


(c) ARC-Challenge


(f) SciQ

Figure 4: Zero-Shot Performance: The horizontal axis is a number of parameters and the vertical axis is accuracy.


Figure 5: Increasing context length contributes to lower test loss on the Pile (Gao et al., 2020).

- Further improving RWKV computational efficiency by applying parallel scan in the (w k v_{t}) step to reduce the computational cost to (O(B \log (T) d)).

- Investigating the application of RWKV to encoder-decoder architectures and potential replacement of cross-attention mechanism. This could have applicability seq 2 seq or multimodal settings, enhancing efficiency both in training and inference.

- Leveraging RWKV's state (or context) for interpretability, predictability in sequence data and safety. Manipulating the hidden state could also guide behavior and allow greater customizability through prompt tuning.

- Exploring fine-tuned models in specific settings for enhanced interaction with humans (Ouyang et al., 2022). Particularly interest- ing would be the performance under different datasets and specific use cases.

- Adapting parameter-efficient fine-tuning methods such as LoRA (Hu et al., 2022) and characterizing behavior under different quantization schemes for the proposed architecture

\section{Conclusions}

We introduced RWKV, a new approach to RNN models exploiting the potential of time-based mixing components. RWKV introduces several key strategies which allow it to capture locality and long-range dependencies, while addressing limitations of current architectures by: (1) replacing the quadratic (QK) attention by a scalar formulation with linear cost, (2) reformulating recurrence and sequential inductive biases to unlock efficient training parallelization and efficient inference, and (3) enhancing training dynamics using custom initializations.

We benchmark the proposed architecture in a wide variety of NLP tasks and show comparable performance to SoTA with reduced cost. Further experiments on expressivity, interpretability, and scaling showcase the model capabilities and draw parallels in behavior between RWKV and other LLMs.

RWKV opens a new door to scalable and efficient architectures to model complex relation- ships in sequential data. While many alternatives to Transformers have been proposed with similar claims, ours is the first to back up those claims with pretrained models with tens of billions of parameters.

\section{Limitations}

While our proposed RWKV model has demonstrated promising results regarding training and memory efficiency during inference, some limitations should be acknowledged and addressed in future work. First, the linear attention of RWKV leads to significant efficiency gains but still, it may also limit the model's performance on tasks that require recalling minutiae information over very long contexts. This is due to the funneling of information through a single vector representation over many time steps, compared with the full information maintained by the quadratic attention of standard Transformers. In other words, the model's recurrent architecture inherently limits its ability to "look back" at previous tokens, as opposed to traditional self-attention mechanisms. While learned time decay helps prevent the loss of information, it is mechanistically limited compared to full selfattention.

Another limitation of this work is the increased importance of prompt engineering in comparison to standard Transformer models. The linear attention mechanism used in RWKV limits the information from the prompt that will be carried over to the model's continuation. As a result, carefully designed prompts may be even more crucial for the model to perform well on tasks.

\section{Acknowledgements}

We acknowledge EleutherAI and StabilityAI for compute access and technical support in development of RWKV. We also acknowledge the members of the RWKV Discord server for their help and work on further extending the applicability of RWKV to different domains. Finally, we thank Stella Biderman for feedback on the paper.

\section{References}

Alon Albalak, Yi-Lin Tuan, Pegah Jandaghi, Connor Pryor, Luke Yoffe, Deepak Ramachandran, Lise Getoor, Jay Pujara, and William Yang Wang. 2022 FETA: A benchmark for few-sample task transfer in open-domain dialogue. In Proceedings of the 2022 Conference on Empirical Methods in Natural Language Processing, pages 10936-10953, Abu Dhabi, United Arab Emirates. Association for Computational Linguistics.

Jimmy Lei Ba, Jamie Ryan Kiros, and Geoffrey E. Hinton. 2016. Layer normalization.

Shaojie Bai, J. Zico Kolter, and Vladlen Koltun. 2018. An empirical evaluation of generic convolutional and recurrent networks for sequence modeling.

Francesco Barbieri, Jose Camacho-Collados, Luis Espinosa Anke, and Leonardo Neves. 2020. TweetEval: Unified benchmark and comparative evaluation for tweet classification. In Findings of the Association for Computational Linguistics: EMNLP 2020, pages 1644-1650, Online. Association for Computational Linguistics.

Iz Beltagy, Matthew E. Peters, and Arman Cohan. 2020. Longformer: The long-document transformer. (\operatorname{arXiv}: 2004.05150).

Stella Biderman, Hailey Schoelkopf, Quentin Anthony, Herbie Bradley, Kyle O'Brien, Eric Hallahan, Mohammad Aflah Khan, Shivanshu Purohit, USVSN Sai Prashanth, Edward Raff, et al. 2023. Pythia: A suite for analyzing large language models across training and scaling. arXiv preprint (\operatorname{arXiv} 22304.01373).

Yonatan Bisk, Rowan Zellers, Ronan Le Bras, Jianfeng Gao, and Yejin Choi. 2020. Piqa: Reasoning about physical commonsense in natural language. In Thirty-Fourth AAAI Conference on Artificial Intelligence.

Sid Black, Leo Gao, Phil Wang, Connor Leahy, and Stella Biderman. 2022. Gpt-neo: Large scale autoregressive language modeling with mesh-tensorflow, 2021. URL: https://doi. org/l0.5281/zenodo, 5297715.

James Bradbury, Stephen Merity, Caiming Xiong, and Richard Socher. 2017. Quasi-recurrent neural networks. In ICLR.

Tom Brown, Benjamin Mann, Nick Ryder, Melanie Subbiah, Jared D Kaplan, Prafulla Dhariwal, Arvind Neelakantan, Pranav Shyam, Girish Sastry, Amanda Askell, et al. 2020. Language models are few-shot learners. Advances in neural information processing systems, 33:1877-1901.

Aydar Bulatov, Yuri Kuratov, and Mikhail S. Burtsev. 2023. Scaling transformer to (1 m) tokens and beyond with rmt.

Aydar Bulatov, Yury Kuratov, and Mikhail Burtsev. 2022. Recurrent memory transformer. Advances in Neural Information Processing Systems, 35:1107911091. A. P. Sarath Chandar, Chinnadhurai Sankar, Eugene Vorontsov, Samira Ebrahimi Kahou, and Yoshua Bengio. 2019. Towards non-saturating recurrent units for modelling long-term dependencies. In AAAI Conference on Artificial Intelligence.

Krzysztof Choromanski, Valerii Likhosherstov, David Dohan, Xingyou Song, Andreea Gane, Tamas Sarlos, Peter Hawkins, Jared Davis, Afroz Mohiuddin, Lukasz Kaiser, David Belanger, Lucy Colwell, and Adrian Weller. 2020. Rethinking attention with performers.

Junyoung Chung, Caglar Gulcehre, KyungHyun Cho, and Yoshua Bengio. 2014. Empirical evaluation of gated recurrent neural networks on sequence modeling. In NIPS 2014 Deep Learning and Representation Learning Workshop.

Peter Clark, Isaac Cowhey, Oren Etzioni, Tushar Khot, Ashish Sabharwal, Carissa Schoenick, and Oyvind Tafjord. 2018. Think you have solved question answering? try arc, the ai 2 reasoning challenge. In arXiv:1803.05457.

Karl Cobbe, Vineet Kosaraju, Mohammad Bavarian, Jacob Hilton, Reiichiro Nakano, Christopher Hesse, and John Schulman. 2021. Training verifiers to solve math word problems. In arXiv, volume (abs / 2110.14168).

Tri Dao, Daniel Y Fu, Stefano Ermon, Atri Rudra, and Christopher Re. 2022a. Flashattention: Fast and memory-efficient exact attention with IO-awareness In Advances in Neural Information Processing Systems.

Tri Dao, Daniel Y Fu, Khaled K Saab, Armin W Thomas, Atri Rudra, and Christopher Ré. 2022b. Hungry hungry hippos: Towards language modeling with state space models. arXiv preprint (\operatorname{arXiv}: 2212.14052).

Dorottya Demszky, Dana Movshovitz-Attias, Jeongwoo Ko, Alan S. Cowen, Gaurav Nemade, and Sujith Ravi. 2020. Goemotions: A dataset of finegrained emotions. In Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics, ACL 2020, Online, July 5-10, 2020, pages 4040-4054. Association for Computational Linguistics.

Leo Gao, Stella Biderman, Sid Black, Laurence Golding, Travis Hoppe, Charles Foster, Jason Phang, Horace He, Anish Thite, Noa Nabeshima, et al. 2020. The pile: An (800 gb) dataset of diverse text for language modeling. arXiv preprint arXiv:2101.00027.

Albert Gu, Karan Goel, and Christopher Ré. 2022. Efficiently modeling long sequences with structured state spaces. In The International Conference on Learning Representations (ICLR).

Albert Gu, Çaglar Gülçehre, Tom Le Paine, Matthew W. Hoffman, and Razvan Pascanu. 2019. Improving the gating mechanism of recurrent neural networks. ArXiv, abs/1910.09890. Mandy Guo, Joshua Ainslie, David C Uthus, Santiago Ontanon, Jianmo Ni, Yun-Hsuan Sung, and Yinfei Yang. 2022. Longt5: Efficient text-to-text transformer for long sequences. In Findings of the Association for Computational Linguistics: NAACL 2022, pages 724-736.

Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun. 2016. Identity mappings in deep residual networks.

Dan Hendrycks, Collin Burns, Steven Basart, Andy Zou, Mantas Mazeika, Dawn Song, and Jacob Steinhardt. 2021. Measuring massive multitask language understanding. In International Conference on Learning Representations.

Sepp Hochreiter. 1998. The vanishing gradient problem during learning recurrent neural nets and problem solutions. International Journal of Uncertainty, Fuzziness and Knowledge-Based Systems, (6(02): 107-116).

Sepp Hochreiter and Jürgen Schmidhuber. 1997. Long short-term memory. Neural Computation, (9(8): 1735-1780).

Jordan Hoffmann, Sebastian Borgeaud, Arthur Mensch, Elena Buchatskaya, Trevor Cai, Eliza Rutherford, Diego de Las Casas, Lisa Anne Hendricks, Johannes Welbl, Aidan Clark, Tom Hennigan, Eric Noland, Katie Millican, George van den Driessche, Bogdan Damoc, Aurelia Guy, Simon Osindero, Karen Simonyan, Erich Elsen, Jack W. Rae, Oriol Vinyals, and Laurent Sifre. 2022. Training compute-optimal large language models.

Edward J Hu, yelong shen, Phillip Wallis, Zeyuan Allen-Zhu, Yuanzhi Li, Shean Wang, Lu Wang, and Weizhu Chen. 2022. LoRA: Low-rank adaptation of large language models. In International Conference on Learning Representations.

Hassan Ismail Fawaz, Germain Forestier, Jonathan Weber, Lhassane Idoumghar, and Pierre-Alain Muller. 2019. Deep learning for time series classification: a review. Data mining and knowledge discovery, 33(4):917-963.

Andrew Jaegle, Felix Gimeno, Andy Brock, Oriol Vinyals, Andrew Zisserman, and Joao Carreira. 2021. Perceiver: General perception with iterative attention. In International conference on machine learning, pages 4651-4664. PMLR.

Hanhwi Jang, Joonsung Kim, Jae-Eon Jo, Jaewon Lee, and Jangwoo Kim. 2019. Mnnfast: A fast and scalable system architecture for memory-augmented neural networks. In Proceedings of the 46th International Symposium on Computer Architecture, pages 250-263.

Matt Gardner Johannes Welbl Nelson F. Liu. 2017. Crowdsourcing multiple choice science questions. In DOI:10.18653/v1/W17-4413. Mandar Joshi, Eunsol Choi, Daniel S. Weld, and Luke Zettlemoyer. 2017. Triviaqa: A large scale distantly supervised challenge dataset for reading comprehension. In (A C L).

John Jumper, Richard Evans, Alexander Pritzel, Tim Green, Michael Figurnov, Olaf Ronneberger, Kathryn Tunyasuvunakool, Russ Bates, Augustin Žídek, Anna Potapenko, and et al. 2021. Highly accurate protein structure prediction with alphafold. Nature, 596(7873):583-589.

Sekitoshi Kanai, Yasuhiro Fujiwara, and Sotetsu Iwamura. 2017. Preventing gradient explosions in gated recurrent units. In NIPS.

Jared Kaplan, Sam McCandlish, Tom Henighan, Tom B Brown, Benjamin Chess, Rewon Child, Scott Gray, Alec Radford, Jeffrey Wu, and Dario Amodei. 2020. Scaling laws for neural language models. arXiv preprint arXiv:2001.08361.

Angelos Katharopoulos, Apoorv Vyas, Nikolaos Pappas, and François Fleuret. 2020. Transformers are rnns: Fast autoregressive transformers with linear attention. In International Conference on Machine Learning, pages 5156-5165. PMLR.

Nikita Kitaev, L. Kaiser, and Anselm Levskaya. 2020. Reformer: The efficient transformer. ArXiv, (abs / 2001.04451).

Jan Kocoń, Igor Cichecki, Oliwier Kaszyca, Mateusz Kochanek, Dominika Szydło, Joanna Baran, Julita Bielaniewicz, Marcin Gruza, Arkadiusz Janz, Kamil Kanclerz, Anna Kocoń, Bartłomiej Koptyra, Wiktoria Mieleszczenko-Kowszewicz, Piotr Miłkowski, Marcin Oleksy, Maciej Piasecki, Łukasz Radliński, Konrad Wojtasik, Stanisław Woźniak, and Przemysław Kazienko. 2023. Chatgpt: Jack of all trades, master of none.

Jan Kocoń, Piotr Miłkowski, and Monika ZaśkoZielińska. 2019. Multi-level sentiment analysis of polemo 2.0: Extended corpus of multi-domain consumer reviews. In Proceedings of the (23 rd) Conference on Computational Natural Language Learning (CoNLL), pages 980-991.

Phong Le and Willem Zuidema. 2016. Quantifying the vanishing gradient and long distance dependency problem in recursive neural networks and recursive lstms. In Proceedings of the 1st Workshop on Representation Learning for NLP, pages 87-93.

Tao Lei, Yu Zhang, Sida I. Wang, Hui Dai, and Yoav Artzi. 2018. Simple recurrent units for highly parallelizable recurrence. In Proceedings of the 2018 Conference on Empirical Methods in Natural Language Processing, pages 4470-4481, Brussels, Belgium. Association for Computational Linguistics.

Hanxiao Liu, Zihang Dai, David R. So, and Quoc V. Le. 2021. Pay attention to mlps. Xuezhe Ma, Xiang Kong, Sinong Wang, Chunting Zhou, Jonathan May, Hao Ma, and Luke Zettlemoyer. 2021. Luna: Linear unified nested attention. Advances in Neural Information Processing Systems, (34: 2441-2453).

Xuezhe Ma, Chunting Zhou, Xiang Kong, Junxian He, Liangke Gui, Graham Neubig, Jonathan May, and Luke Zettlemoyer. 2023. Mega: Moving average equipped gated attention. In ICLR.

Eric Martin and Chris Cundy. 2017. Parallelizing linear recurrent neural nets over sequence length. ArXiv, abs/1709.04057.

Kevin Meng, David Bau, Alex Andonian, and Yonatan Belinkov. 2022. Locating and editing factual associations in GPT. Advances in Neural Information Processing Systems, 36.

Todor Mihaylov, Peter Clark, Tushar Khot, and Ashish Sabharwal. 2018. Can a suit of armor conduct electricity? a new dataset for open book question answering. In (E M N L P).

John Miller and Moritz Hardt. 2018. Stable recurrent models. arXiv: Learning.

Nasrin Mostafazadeh, Nathanael Chambers, Xiaodong He, Devi Parikh, Dhruv Batra, Lucy Vanderwende, Pushmeet Kohli, and James Allen. 2016. A corpus and cloze evaluation for deeper understanding of commonsense stories. In Proceedings of the 2016 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, pages 839-849.

OpenAI. 2022. Introducing chatgpt. https://openai . com/blog/chatgpt.

OpenAI. 2023. Gpt-4 technical report.

Antonio Orvieto, Samuel L Smith, Albert Gu, Anushan Fernando, Caglar Gulcehre, Razvan Pascanu, and Soham De. 2023. Resurrecting recurrent neural networks for long sequences. arXiv preprint (\operatorname{arXiv}: 2303.06349).

Long Ouyang, Jeff Wu, Xu Jiang, Diogo Almeida, Carroll L. Wainwright, Pamela Mishkin, Chong Zhang, Sandhini Agarwal, Katarina Slama, Alex Ray, John Schulman, Jacob Hilton, Fraser Kelton, Luke Miller, Maddie Simens, Amanda Askell, Peter Welinder, Paul Christiano, Jan Leike, and Ryan Lowe. 2022. Training language models to follow in structions with human feedback.

Denis Paperno, Germán Kruszewski, Angeliki Lazaridou, Ngoc Quan Pham, Raffaella Bernardi, Sandro Pezzelle, Marco Baroni, Gemma Boleda, and Raquel Fernandez. 2016. The LAMBADA dataset: Word prediction requiring a broad discourse context. In Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), pages 1525-1534, Berlin, Germany. Association for Computational Linguistics. Razvan Pascanu, Tomas Mikolov, and Yoshua Bengio. 2012. On the difficulty of training recurrent neural networks. In International Conference on Machine Learning.

Adam Paszke, Sam Gross, Francisco Massa, Adam Lerer, James Bradbury, Gregory Chanan, Trevor Killeen, Zeming Lin, Natalia Gimelshein, Luca Antiga, Alban Desmaison, Andreas Köpf, Edward Yang, Zach DeVito, Martin Raison, Alykhan Tejani, Sasank Chilamkurthy, Benoit Steiner, Lu Fang, Junjie Bai, and Soumith Chintala. 2019. Pytorch: An imperative style, high-performance deep learning library.

Michael Poli, Stefano Massaroli, Eric Nguyen, Daniel Y Fu, Tri Dao, Stephen Baccus, Yoshua Bengio, Stefano Ermon, and Christopher Ré. 2023. Hyena hierarchy: Towards larger convolutional language models. arXiv preprint arXiv:2302.10866.

Ofir Press, Noah A. Smith, and Mike Lewis. 2022. Train short, test long: Attention with linear biases enables input length extrapolation. In The Tenth International Conference on Learning Representations, ICLR 2022, Virtual Event, April 25-29, 2022.

Ilan Price, Jordan Gifford-Moore, Jory Flemming, Saul Musker, Maayan Roichman, Guillaume Sylvain, Nithum Thain, Lucas Dixon, and Jeffrey Sorensen. 2020. Six attributes of unhealthy conversations. In Proceedings of the Fourth Workshop on Online Abuse and Harms, pages 114-124, Online. Association for Computational Linguistics.

Markus N. Rabe and Charles Staats. 2022. Selfattention does not need (o\left(n^{2}\right)) memory.

Melissa Roemmele, Cosmin Adrian Bejan, , and Andrew S. Gordon. 2018. Choice of plausible alternatives: An evaluation of commonsense causal reasoning. In (A A A I).

Teven Le Scao, Angela Fan, Christopher Akiki, Ellie Pavlick, Suzana Ilić, Daniel Hesslow, Roman Castagné, Alexandra Sasha Luccioni, François Yvon, Matthias Gallé, et al. 2022. Bloom: A 176bparameter open-access multilingual language model. arXiv preprint arXiv:2211.05100.

Ramsha Siddiqui. 2019. SARCASMANIA: Sarcasm Exposed! http://www.kaggle.com/rmsharks4/ sarcasmania-dataset. [Online; accessed 02February-2023].

David R. So, Wojciech Manke, Hanxiao Liu, Zihang Dai, Noam Shazeer, and Quoc V. Le. 2021. Primer: Searching for efficient transformers for language modeling. CoRR, abs/2109.08668.

Yi Tay, Dara Bahri, Donald Metzler, Da-Cheng Juan, Zhe Zhao, and Che Zheng. 2020. Synthesizer: Rethinking self-attention in transformer models.

Yi Tay, Mostafa Dehghani, Dara Bahri, and Donald Metzler. 2022. Efficient transformers: A survey. ACM Computing Surveys, 55(6):1-28. Ilya O. Tolstikhin, Neil Houlsby, Alexander Kolesnikov, Lucas Beyer, Xiaohua Zhai, Thomas Unterthiner, Jessica Yung, Andreas Steiner, Daniel Keysers, Jakob Uszkoreit, Mario Lucic, and Alexey Dosovitskiy. 2021. Mlp-mixer: An all-mlp architecture for vision. CoRR, abs/2105.01601.

Hugo Touvron, Thibaut Lavril, Gautier Izacard, Xavier Martinet, Marie-Anne Lachaux, Timothée Lacroix, Baptiste Rozière, Naman Goyal, Eric Hambro, Faisal Azhar, Aurelien Rodriguez, Armand Joulin, Edouard Grave, and Guillaume Lample. 2023 Llama: Open and efficient foundation language models.

Aäron van den Oord, Sander Dieleman, Heiga Zen, Karen Simonyan, Oriol Vinyals, Alex Graves, Nal Kalchbrenner, Andrew W. Senior, and Koray Kavukcuoglu. 2016. Wavenet: A generative model for raw audio. ArXiv, abs/1609.03499.

Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez, Łukasz Kaiser, and Illia Polosukhin. 2017. Attention is all you need. In Advances in Neural Information Processing Systems, volume 30. Curran Associates, Inc.

David Vilares and Carlos Gómez-Rodríguez. 2019. Head-qa: A healthcare dataset for complex reasoning. In (A C L).

Alex Wang, Yada Pruksachatkun, Nikita Nangia, Amanpreet Singh, Julian Michael, Felix Hill, Omer Levy, and Samuel Bowman. 2019. Superglue: A stickier benchmark for general-purpose language understanding systems. In Advances in Neural Information Processing Systems, volume 32. Curran Associates, Inc.

Alex Wang, Amanpreet Singh, Julian Michael, Felix Hill, Omer Levy, and Samuel Bowman. 2018. GLUE: A multi-task benchmark and analysis platform for natural language understanding. In Proceedings of the 2018 EMNLP Workshop BlackboxNLP: Analyzing and Interpreting Neural Networks for NLP, pages 353-355, Brussels, Belgium Association for Computational Linguistics.

Sinong Wang, Belinda Z. Li, Madian Khabsa, Han Fang, and Hao Ma. 2020. Linformer: Self-attention with linear complexity.

Zonghan Wu, Shirui Pan, Fengwen Chen, Guodong Long, Chengqi Zhang, and S Yu Philip. 2020. A comprehensive survey on graph neural networks. IEEE transactions on neural networks and learning systems, 32(1):4-24.

Ellery Wulczyn, Nithum Thain, and Lucas Dixon. 2017. Ex machina: Personal attacks seen at scale. In Proceedings of the 26th International Conference on World Wide Web, WWW 2017, Perth, Australia, April 3-7, 2017, pages 1391-1399. ACM. Weihao Yu, Mi Luo, Pan Zhou, Chenyang Si, Yichen Zhou, Xinchao Wang, Jiashi Feng, and Shuicheng Yan. 2022. Metaformer is actually what you need for vision.

Manzil Zaheer, Guru Guruganesh, Kumar Avinava Dubey, Joshua Ainslie, Chris Alberti, Santiago Ontanon, Philip Pham, Anirudh Ravula, Qifan Wang, Li Yang, et al. 2020. Big bird: Transformers for longer sequences. Advances in Neural Information Processing Systems, 33.

Rowan Zellers, Ari Holtzman, Yonatan Bisk, Ali Farhadi, and Yejin Choi. 2019. Hellaswag: Can a machine really finish your sentence? In (A C L).

Rowan Zellers, Ari Holtzman, Yonatan Bisk, Ali Farhadi, and Yejin Choi. 2020. Winogrande: An adversarial winograd schema challenge at scale. In (A C L).

Shuangfei Zhai, Walter Talbott, Nitish Srivastava, Chen Huang, Hanlin Goh, Ruixiang Zhang, and Josh Susskind. 2021. An attention free transformer.

Sheng Zhang, Xiaodong Liu, Jingjing Liu, Jianfeng Gao, Kevin Duh, and Benjamin Van Durme. 2018 Record: Bridging the gap between human and machine commonsense reading comprehension. In arXiv: 1810.12885

Susan Zhang, Stephen Roller, Naman Goyal, Mikel Artetxe, Moya Chen, Shuohui Chen, Christopher Dewan, Mona Diab, Xian Li, Xi Victoria Lin, et al. 2022. Opt: Open pre-trained transformer language models. arXiv preprint arXiv:2205.01068.

\section{A Author Contributions}

Bo Peng Original RWKV idea, original code, performance optimizations, original experiments, and trained RWKV models from (0.1 B) to (14 B).

Eric Alcaide Manuscript (initial draft sections 1, 2; sections 4, 7 and 8; revision and proofreading; final version ). Figures ((2,3,4,7)). Experiments section 6. Appendices D, I. Contributions to Appendix K.

Quentin Anthony Led writing the paper. Manuscript (initial draft sections 1, 2, 3; revision and proofreading; final version).

Zhenyuan Zhang Manuscript (revision and proofreading) Figure 3. Experiments Appendix G. Contributions to Appendices B and K.

Kranthi Kiran GV Manuscript (sections 2 and 5; contributions to section 3; revision and proofreading). Tables 3 and 4. Appendix C. Xiangru Tang Manuscript (sections 2 and 3; contributions to abstract; revision and proofreading). Contributions to Appendix K.

Matteo Grella Manuscript (sections 4.5, 4.6, 8; contributions to sections 1,7 and 9 ; proofreading and revision). Contributions to Appendix B.

Ferdinand Mom Manuscript (contributions to section 1, 2, 4.3, 4.6; proofreading and revision). Contributions to Appendix B.

Atsushi Saito Manuscript (sections 3 and 5; contributions to section 2). Figures 1a, 1b, 1c. Contributions to Appendix (H)

Krishna Sri Ipsit Mantri Figure 4

Rui-Jie Zhu Tables 1 and 5. Experiments for table 5

Peng Zhou Contributions to Table 5 .

Qihang Zhao Manuscript (proofreading and revision). Contributions to Table 5.

Xuzheng He Manuscript (contributions to section 3; proofreading and revision). Contributions to Figures 1, 7. Appendix G. Contributions to appendix (F).

Hayden Lau Manuscript (contributions to section 1; proofreading and revision). Contributions to Appendix K.

Michael Chung Manuscript (contributions to section 4.6; proofreading and revision).

Haowen Hou Figure 8. Appendix E

Jiaming Kong Manuscript (revision and proofreading). Appendix (F).

Johan S. Wind RWKV performance optimizations (CUDA), Contributions to Appendix C.

Jian Zhu Manuscript (section 2; proofreading and revision). Figures 3 and 5 .

Huanqi Cao Manuscript (contributions to 4.2 and 4.3; proofreading and revision). Experiments for Appendix G.

Samuel Arcadinho Contributions to Figures 6, 10, and 11. Contributions to Appendix I.

Xin Cheng Manuscript (proofreading and revision). Contributions to Appendix K, H. Alon Albalak Manuscript (abstract and sections 1,(9 ;) proofreading and revision).

Jan Kocon Manuscript (sections 1; proofreading and revision). Contributions to Appendix J.

Przemysław Kazienko Manuscript (section 6; proofreading and revision). Contributions Appendix J.

Ruichong Zhang Manuscript (proofreading and revision); Contributions to Figure 5 and Appendix (K).

Stanisław Woźniak Appendix J.

Bartlomiej Koptyra Contributions to Appendix J.

\section{B Time-Mixing Block as an RNN Cell}

As stated in 4.3, the RWKV time-mixing block can be formulated as an RNN, as the (W K V) computation can be written in such a recursive form:

\[
\begin{aligned}
a_{0}, b_{0} & =0, \\
w k v_{t} & =\frac{a_{t-1}+e^{u+k_{t}} v_{t}}{b_{t-1}+e^{u+k_{t}}}, \\
a_{t} & =e^{-w} a_{t-1}+e^{k_{t}} v_{t}, \\
b_{t} & =e^{-w} b_{t-1}+e^{k_{t}} .
\end{aligned}
\]

The dataflow of the RNN-like time-mixing is shown in Fig. 7, where the hidden states (h) is the numerator-denominator tuple ((a, b)).


Figure 7: RWKV time-mixing block formulated as an RNN cell. Color codes: yellow ((\mu)) denotes the token shift, red (1) denotes the denominator, blue (2) denotes the numerator, pink (3) denotes the fraction computations in 14. (h) denotes the numerator-denominator tuple ((a, b)).

To avoid overflow in calculating (e^{k_{t}}), a numerical trick is used in the official implementation. Note that

\[
\begin{aligned}
a_{1} & =e^{-w} a_{0}+e^{k_{0}} v_{0}=e^{k_{0}} v_{0}, \\
b_{1} & =e^{-w} b_{0}+e^{k_{0}}=e^{k_{0}},
\end{aligned}
\]

and we set (a_{1}^{\prime}=v_{0}, b_{1}^{\prime}=1, p_{0}=k_{0}), where (p_{t-1}) stores the shared exponents of (a_{t}) and (b_{t}). Now the above recursion can be converted into a numerical safe version, for each time step (t>1) :

\[
\begin{aligned}
q & :=\max \left(p_{t-1}, u+k_{t}\right), \\
a_{t}^{*} & =e^{p_{t-1}-q} a_{t-1}^{\prime}+e^{u+k_{t}-q} v_{t}, \\
b_{t}^{*} & =e^{p_{t-1}-q} b_{t-1}^{\prime}+e^{u+k_{t}-q}, \\
w k v_{t} & =\frac{a_{t}^{*}}{b_{t}^{*}} .
\end{aligned}
\]

The update to (a_{t}^{\prime}, b_{t}^{\prime}) and their shared exponent are also carried out in similar fashion:

\[
\begin{aligned}
q & :=\max \left(p_{t-1}-w, k_{t}\right), \\
a_{t}^{\prime} & =e^{p_{t-1}-w-q} a_{t-1}^{\prime}+e^{k_{t}-q} v_{t}, \\
b_{t}^{\prime} & =e^{p_{t-1}-w-q} b_{t-1}^{\prime}+e^{k_{t-q}}, \\
p_{t} & =q .
\end{aligned}
\]

\section{Parameter and FLOP Count for the RWKV Models}

The following section provides an overview of the different RWKV model architectures along with their respective parameter and FLOP counts in Table 2 .

\begin{tabular}{ccccc}
\hline Name & Layers & Model Dimension & Parameters & FLOPs per token \\
\hline (169 M) & 12 & 768 & (1.693 \times 10^{8}) & (2.613 \times 10^{8}) \\
(430 M) & 24 & 1024 & (4.304 \times 10^{8}) & (7.573 \times 10^{8}) \\
(1.5 B) & 24 & 2048 & (1.515 \times 10^{9}) & (2.823 \times 10^{9}) \\
3B & 32 & 2560 & (2.985 \times 10^{9}) & (5.710 \times 10^{9}) \\
7B & 32 & 4096 & (7.393 \times 10^{9}) & (1.437 \times 10^{10}) \\
(14 B) & 40 & 5120 & (1.415 \times 10^{10}) & (2.778 \times 10^{10}) \\
\hline
\end{tabular}

Table 2: RWKV model architectures and associated FLOP counts

The number of parameters for each model is computed using the formula: \#parameters (=) (2 V D+13 D^{2} L+D(11 L+4)) where (V=50277) is the vocabulary size, (D) represents the Model Dimension and (L) corresponds to the number of layers.

FLOPs is for a forward pass for one token. It was calculated as (6\left(V D+13 D^{2} L\right)), which is the twice (add and multiply) the number of parameters in linear layers. The backwards pass FLOPs can be approximated as twice that of the forward pass. So the total is (6\left(V D+13 D^{2} L\right)) per token for training ( (3 x) fw FLOPs). It is noteworthy that FLOPs are independent of the context length, unlike regular transformers. The FLOP approximations in this paper are in line with the methodology used by Kaplan et al. (2020).

Alternative approximations for FLOPs include doubling the parameters which yields similar results within (2 %) for (14 B) and a (30 %) discrepancy for (169 M) variant. Another approximation is based on the number of non-embedding parameters multiplied by 2 . This gives (2\left(V D+13 D^{2} L+D(11 L+\right.) 4)) resulting in (1.6 %) more FLOPs for (14 B) model and (8 %) more FLOPs for (169 M) model.

\section{Parameter initializations}

We describe the specific parameter initializations below and motivate the design choices. Parameters belonging to residual blocks are often adjusted by layer depth and total number of layers. Let \# denote the vocabulary size, (s) denote the embedding dimension, (d) denote the hidden size (we use (d=4 s), L) the number of layers, (l) the layer index (from 0 to L-1), we use the following initializations:

- Embeddings are initialized to (U ( \pm 1 e-4)) as explained in 4.7

- For the channel-mixing blocks (11), (\mu_{k_{i}}) and (\mu_{r_{i}}) are initialized to (\left(\frac{i}{s}\right)^{1-\frac{l}{L}})

- For the time-mixing blocks (16), initializations are (\mu_{k_{i}}=\left(\frac{i}{s}\right)^{1-\frac{l}{L}}, \mu_{v_{i}}=\left(\frac{i}{s}\right)^{1-\frac{l}{L}}+\frac{0.3 l}{L-1}) and (\mu_{r_{i}}=0.5\left(\frac{i}{s}\right)^{1-\frac{l}{L}})

- (w_{i}) (14), also known as "time decay", is initialized to (-5+8 \cdot\left(\frac{i}{d-1}\right)^{0.7+\frac{1.3 l}{L-1}}). Intuitively, it is the discount factor applied to previous tokens over time.

- (u_{i}) (14), also known as "bonus", is set to (0.5(((i+1) \bmod 3)-1)+\log 0.3). It is the special weighting applied to the current token in equation 14. The alternating zigzag pattern initially creates subtle variations in the tensor elements, which are intended to help the model treat different dimensions of the embedding distinctively.

- (W_{o}) (15) (time-mixing) and (W_{v}) (channelmixing) are initialized to (N \left(0, \sqrt{\frac{d}{s}}=2\right))

- All (W_{r}, W_{k}, W_{v}) weights are initialized to 0 so the model can start learning from the beginning without noisy signals. - All LayerNorm weights start from 1 and biases from 0.

\section{E Small Init Embedding}

This section presents experimental validation of small initialization embedding. The experimental setup is as follows. In the baseline configuration, the parameters are initialized using a normal distribution with a mean of 0.0 and a standard deviation of 0.02 , which is a commonly used initialization method in models like BERT and GPT. On the other hand, in the small initialization of the embedding (small init emb) experiment, the parameters are initialized using a uniform distribution with a range of 1e-4, which is slightly different from RWKV where a normal distribution with a standard deviation of 1e-4 is used. However, this difference is negligible and does not affect our conclusions. The experiments were conducted with a batch size of 400 . As depicted in the figure 8 , the loss curve for the small init emb exhibits a faster rate of decrease and convergence compared to the traditional initialization using a normal distribution.


Figure 8: Effect of small initialization embedding.

\section{F Gradient Stability in RWKV}

In this section, we present a mathematical description of the gradient stability property in RWKV, focusing specifically on the time-mixing block. By gradient stability we mean that if the inputs (x_{t}) are bounded and the model parameters are fixed, then the gradients with respect to (W_{k}) and (W_{v}) are uniformly bounded for all (T) (thus not exploding). Consequently, we can control the amount each (x_{t}) contributes to the gradient at (T) in a naturally decaying fashion by the weight decay mechanism (w) (thus not vanishing unless desired).

First, we make the simplification that there are no token shifts, this will not affect the final conclusion. In this scenario, (w k v_{T}) can be written as

\[
w k v_{T}=\frac{\sum_{t=1}^{T} K_{t}^{e} v_{t}}{\sum_{t=1}^{T} K_{t}^{e}}= E \left(v_{t}\right)=\frac{ S \left(v_{t}\right)}{ S (1)}
\]

where

\[
\begin{gathered}
v_{t}=W_{v} x_{t}, \quad \frac{\partial\left(v_{t}\right)_{i}}{\partial\left(W_{v}\right)_{i, j}}=\left(x_{t}\right)_{j}, \\
K_{t}^{e}=e^{W_{k} x_{t}+w_{T, t}}, \quad \frac{\partial\left(K_{t}^{e}\right)_{i}}{\partial\left(W_{k}\right)_{i, j}}=\left(x_{t}\right)_{j}\left(K_{t}^{e}\right)_{i},
\end{gathered}
\]

and (S (\cdot)) and (E (\cdot)) are shorthand for denoting sums and averages over weights (K_{t}^{e}).

The loss function at position (T) can be written as

\[
L_{T}=l\left(f\left(w k v_{T}\right), y_{T}\right)
\]

Because (w k v_{T}) relates to (\left(W_{k}\right)_{i, j}) and (\left(W_{v}\right)_{i, j}) only through the (i)-th channel (\left(w k v_{T}\right)_{i}), we have

\[
\frac{\partial L_{T}}{\partial\left(W_{v}\right)_{i, j}}=\frac{\partial L_{T}}{\partial\left(w k v_{T}\right)_{i}} \frac{\partial\left(w k v_{T}\right)_{i}}{\partial\left(W_{v}\right)_{i, j}}
\]

The first part of above equation contains trivial operations like output layers, and other layers of time-mixing, which can be proven inductively. The second part of above equation can be bounded as

\[
\begin{aligned}
\left|\frac{\partial\left(w k v_{T}\right)_{i}}{\partial\left(W_{v}\right)_{i, j}}\right| & =\left|\frac{\partial E _{i}\left[\left(v_{t}\right)_{i}\right]}{\partial\left(W_{v}\right)_{i, j}}\right| \\
& =\left| E _{i}\left[\left(x_{t}\right)_{j}\right]\right| \leq \max _{t}\left|\left(x_{t}\right)_{j}\right|,
\end{aligned}
\]

which is irrelevant to (T). Similarly,

\[
\begin{aligned}
\frac{\partial\left(w k v_{T}\right)_{i}}{\partial\left(W_{k}\right)_{i, j}} & =\partial \frac{ S _{i}\left[\left(v_{t}\right)_{i}\right]}{ S _{i}(1)} / \partial\left(W_{k}\right)_{i, j} \\
& =\frac{ S _{i}\left[\left(x_{t}\right)_{j}\left(v_{t}\right)_{i}\right]}{ S _{i}(1)}-\frac{ S _{i}\left[\left(x_{t}\right)_{j}\right] S _{i}\left[\left(v_{t}\right)_{i}\right]}{ S _{i}(1)^{2}} \\
& = E _{i}\left[\left(x_{t}\right)_{j}\left(v_{t}\right)_{i}\right]- E _{i}\left[\left(x_{t}\right)_{j}\right] E _{i}\left[\left(v_{t}\right)_{i}\right] \\
& =\operatorname{cov}_{i}\left(\left(x_{t}\right)_{j},\left(v_{t}\right)_{i}\right)
\end{aligned}
\]

can also be bounded. Note that (w k v) 's softmax operation contains at least two non-zero terms ( (u) and (w) ), so the above "covariance" will not degenerate into 0 .

Figure 9: Model behavior visualizations of the RWKV model.

\section{G Model Behavior Visualization}

In Figure 9, we present visualizations of some behavior of the RWKV model.

The top plot illustrates the time decays (\left(e^{-w}\right)) in each layer of the RWKV-169M model, sorted along the channel axis. Notably, several decays in the last layers are very close or equal to one, implying that certain information is preserved and propagated throughout the model's temporal context. Meanwhile, many decays in the initial layer are close to zero, which corresponds to local operations in (w k v) (14), likely to be associated with tasks such as text parsing or lexical analysis. (Note that the local operations in (w k v) is due to the extra parameter (u), when (e^{-w}) is degenerated into 0 .) These patterns of time decays are partly learned, but also come from parameter initialization as it speeds up training.

The bottom plot shows the information retrieval and propagation path in the RWKV-430M model. The experiment follows the causal trace method introduced by Meng et al. (2022), where we

1. Run the model once, and record all states and activation of each layer during the computation;

2. Corrupt the input embeddings of the subject using noise ("The Eiffel Tower" in this example); 3. Restore the states and activation of a certain layer at a certain token during the computation, and record the log-probability of the model outputting the correct answer ("Paris").

Unlike transformers, RWKV relies on recursive propagation of information in the time dimension. In this case, the fact that "the Eiffel Tower is located in Paris" is retrieved in layer 4. It is then passed down to the subsequent layers. In layer 20, mostly, the information is propagated through time until reaching where it is needed. Finally, it is passed down to the last layer for outputting the answer.

\section{H Evaluation Details}

The results for following tasks are in Table 3 and 4 .

Tasks:

- LAMBADA (Paperno et al., 2016). A benchmark dataset that evaluates the model's contextual reasoning and language comprehension abilities by presenting context-target pairs, where the objective is to predict the most probable target token.

- PIQA (Bisk et al., 2020). A benchmark for the task of physical common sense reasoning, which consists of a binary choice task that can be better understood as a set of two pairs, namely (Goal, Solution).

- HellaSwag (Zellers et al., 2019) A novel benchmark for commonsense Natural Language Inference (NLI) which is build by adversarial filtering against transformer models.

- Winogrande (Zellers et al., 2020) A dataset designed to evaluate the acquisition of common sense reasoning by neural language models, aiming to determine whether we are accurately assessing the true capabilities of machine common sense.

- StoryCloze (Mostafazadeh et al., 2016) A benchmark to present a novel approach to assess comprehension of narratives, narrative generation, and script acquisition, focusing on commonsense reasoning.

- ARC Challenge (Clark et al., 2018) A dataset designed for multiple-choice question answering, encompassing science exam questions ranging from third grade to ninth grade.

- ARC Easy An easy subset of ARC.

- HeadQA (Vilares and Gómez-Rodríguez, 2019) A benchmark consisting of graduatelevel questions encompassing various fields such as medicine, nursing, biology, chemistry, psychology, and pharmacology.

- OpenBookQA (Mihaylov et al., 2018) A QA dataset to evaluate human comprehension of a subject by incorporating open book facts, scientific knowledge, and perceptual common sense, drawing inspiration from open book exams.

- SciQ (Johannes Welbl Nelson F. Liu, 2017) A multiple-choice QA dataset which was created using an innovative approach to gather well-crafted multiple-choice questions that are focused on a specific domain.

- TriviaQA (Joshi et al., 2017) A QA-IR dataset which is constituted of triples of questions, answers, supporting evidence, and independently collected evidence documents, with an average of six documents per question for reliable sources.

- ReCoRD (Zhang et al., 2018) A benchmark for evaluating commonsense reasoning in reading comprehension by generating queries from CNN/Daily Mail news articles and requiring text span answers from corresponding summarizing passages.

- COPA (Roemmele et al., 2018) A dataset to evaluate achievement in open-domain commonsense causal reasoning.

- MMMLU (Hendrycks et al., 2021) A multitask dataset for 57 tasks containing elementary mathematics, US history, computer science, law, etc.

\section{Inference results}

Figures 10 and 11 illustrate, respectively, the results on time (s) and memory (RAM, VRAM) requirements for LLM inference in float32 precision. We benchmark the following model families and sizes:

- RWKV: 169m, 430m, 1.4b, 3b, 7b, 14b

- Bloom (Scao et al., 2022): 560m, 1b, 3b

- OPT (Zhang et al., 2022): 125m, 350m, 1.3b, (2.7 b, 6.7 b, 13 b)

- GPT-Neo (Black et al., 2022): 125m, 1.3b, (2.7 b)

- Pythia (Biderman et al., 2023): 160m, 410m, (1.4 b, 2.8 b, 6.7 b, 12 b)

Missing models in are due to Out Of Memory (OOM) errors. A comparison at 512 tokens is shown in Figure 11 as some large transformer models produced an OOM when inferencing longer se- 

\begin{tabular}{|c|c|c|c|c|c|c|c|c|}
\hline Model & (\begin{array}{l}\text { Params } \\
\text { B }\end{array}) & (\begin{array}{l}\text { PIQA } \\
\text { acc }\end{array}) & (\begin{array}{l}\text { StoryCloze } \\
\text { acc }\end{array}) & (\begin{array}{l}\text { HellaSwag } \\
\text { acc_norm }\end{array}) & (\begin{array}{l}\text { WinoGrande } \\
\text { acc }\end{array}) & (\begin{array}{l}\text { ARC-e } \\
\text { acc }\end{array}) & (\begin{array}{l}\text { ARC-c } \\
\text { acc_norm }\end{array}) & (\begin{array}{l}\text { OBQA } \\
\text { acc_norm }\end{array}) \\
\hline RWKV-4 & 0.17 & 65.07 & 58.79 & 32.26 & 50.83 & 47.47 & 24.15 & 29.60 \\
\hline Pythia & 0.16 & 62.68 & 58.47 & 31.63 & 52.01 & 45.12 & 23.81 & 29.20 \\
\hline GPT-Neo & 0.16 & 63.06 & 58.26 & 30.42 & 50.43 & 43.73 & 23.12 & 26.20 \\
\hline RWKV-4 & 0.43 & 67.52 & 63.87 & 40.90 & 51.14 & 52.86 & 25.17 & 32.40 \\
\hline Pythia & 0.40 & 66.70 & 62.64 & 39.10 & 53.35 & 50.38 & 25.77 & 30.00 \\
\hline GPT-Neo & 0.40 & 65.07 & 61.04 & 37.64 & 51.14 & 48.91 & 25.34 & 30.60 \\
\hline RWKV-4 & 1.5 & 72.36 & 68.73 & 52.48 & 54.62 & 60.48 & 29.44 & 34.00 \\
\hline Pythia & 1.4 & 71.11 & 67.66 & 50.82 & 56.51 & 57.74 & 28.58 & 30.80 \\
\hline GPT-Neo & 1.4 & 71.16 & 67.72 & 48.94 & 54.93 & 56.19 & 25.85 & 33.60 \\
\hline RWKV-4 & 3.0 & 74.16 & (7 0 . 7 1) & 59.89 & 59.59 & 65.19 & 33.11 & 37.00 \\
\hline Pythia & 2.8 & 73.83 & 70.71 & 59.46 & 61.25 & 62.84 & 32.25 & 35.20 \\
\hline GPT-Neo & 2.8 & 72.14 & 69.54 & 55.82 & 57.62 & 61.07 & 30.20 & 33.20 \\
\hline RWKV-4 & 7.4 & 76.06 & 73.44 & 65.51 & 61.01 & 67.80 & 37.46 & 40.20 \\
\hline Pythia & 6.9 & 74.54 & 72.96 & 63.92 & 61.01 & 66.79 & 35.07 & 38.00 \\
\hline GPT-J & 6.1 & 75.41 & 74.02 & 66.25 & 64.09 & 66.92 & 36.60 & 38.20 \\
\hline RWKV-4 & 14.2 & (7 7 . 4 8) & 76.06 & 70.65 & 63.85 & 70.24 & 38.99 & 41.80 \\
\hline GPT-level (^{*}) & 14.2 & 76.49 & 74.97 & 68.72 & 65.14 & 70.77 & 37.99 & 39.27 \\
\hline Pythia (c.f.) & 11.8 & 75.90 & 74.40 & 67.38 & 64.72 & 69.82 & 36.77 & 38.80 \\
\hline GPT-NeoX (c.f.) & 20.6 & 77.69 & 76.11 & 71.42 & 65.98 & 72.69 & 40.44 & 40.20 \\
\hline
\end{tabular}

Table 3: Zero-Shot Performance of the model on Common Sense Reasoning Tasks. ({ }^{*}) Interpolation of Pythia and GPT-Neo models

\begin{tabular}{lllllllll}
\hline Model & (\begin{array}{l}\text { Params } \\
\text { B }\end{array}) & (\begin{array}{l}\text { LAMBADA } \\
\text { ppl }\end{array}) & (\begin{array}{l}\text { LAMBADA } \\
\text { acc }\end{array}) & (\begin{array}{l}\text { headQA } \\
\text { acc_norm }\end{array}) & (\begin{array}{l}\text { sciq } \\
\text { acc }\end{array}) & (\begin{array}{l}\text { triviaQA } \\
\text { acc }\end{array}) & (\begin{array}{l}\text { ReCoRD } \\
\text { em }\end{array}) & (\begin{array}{l}\text { COPA } \\
\text { acc }\end{array}) \\
\hline RWKV-4 & 0.17 & (2 9 . 3 3) & 32.99 & 25.78 & (7 7 . 5 0) & 1.26 & 62.03 & (6 6 . 0 0) \\
Pythia & 0.16 & 24.38 & (3 8 . 9 7) & (2 5 . 8 2) & 76.50 & (1 . 3 1) & (6 6 . 3 2) & 62.00 \\
GPT-Neo & 0.16 & 30.27 & 37.36 & 25.16 & 76.60 & 1.18 & 64.92 & 64.00 \\
\hline RWKV-4 & 0.43 & 13.04 & 45.16 & (2 7 . 3 2) & 80.30 & (2 . 3 5) & 70.48 & 65.00 \\
Pythia & 0.40 & (1 1 . 5 8) & (5 0 . 4 4) & 25.09 & (8 1 . 5 0) & 2.03 & (7 5 . 0 5) & (6 7 . 0 0) \\
GPT-Neo & 0.40 & 13.88 & 47.29 & 26.00 & 81.10 & 1.38 & 73.79 & 65.00 \\
\hline RWKV-4 & 1.5 & 7.04 & 56.43 & (2 7 . 6 4) & 85.00 & (5 . 6 5) & 76.97 & (7 7 . 0 0) \\
Pythia & 1.4 & (6 . 5 8) & (6 0 . 4 3) & 27.02 & (8 5 . 5 0) & 5.52 & (8 1 . 4 3) & 73.00 \\
GPT-Neo & 1.4 & 7.5 & 57.25 & 27.86 & 86.00 & 5.24 & 80.62 & 69.00 \\
\hline RWKV-4 & 3.0 & 5.25 & 63.96 & 28.45 & 86.50 & (1 1 . 6 8) & 80.87 & (8 2 . 0 0) \\
Pythia & 2.8 & (4 . 9 3) & (6 5 . 3 6) & (2 8 . 9 6) & (8 7 . 7 0) & 9.63 & 85.10 & 77.00 \\
GPT-Neo & 2.8 & 5.63 & 62.22 & 27.17 & 89.30 & 4.82 & (8 3 . 8 0) & 80.00 \\
\hline RWKV-4 & 7.4 & 4.38 & 67.18 & (3 1 . 2 2) & 88.80 & (1 8 . 3 0) & 83.68 & (8 5 . 0 0) \\
Pythia & 6.9 & 4.3 & (6 7 . 9 8) & 28.59 & 90.00 & 15.42 & 86.44 & (8 5 . 0 0) \\
GPT-J & 6.1 & (4 . 1) & 68.31 & 28.67 & (9 1 . 5 0) & 16.74 & (8 7 . 7 1) & 83.00 \\
\hline RWKV-4 & 14.2 & (3 . 8 6) & 70.83 & (3 2 . 6 4) & 90.40 & (2 4 . 5 8) & 85.67 & (8 5 . 0 0) \\
GPT-level & 14.2 & 3.81 & (7 0 . 9 4) & 31.03 & (9 2 . 2 0) & 22.37 & 87.89 & 82.66 \\
\hline Pythia (c.f.) & 11.8 & 3.89 & 70.44 & 30.74 & 91.80 & 20.57 & 87.58 \\
GPT-NeoX (c.f.) & 20.6 & 3.64 & 71.94 & 31.62 & 93.00 & 25.99 & 88.52 & 82.00 \\
\hline
\end{tabular}

Table 4: Zero-Shot Performance of various models on different tasks. * Interpolation of Pythia and GPT-Neo models 

\begin{tabular}{llllllll}
\hline Method & (L) & (d) & (T) & Train bpc & Test bpc & Time Complexity & Space Complexity \\
\hline Transformer & 12 & 512 & 1024 & 0.977 & 1.137 & (O\left(T^{2} d\right)) & (O\left(T^{2}+T d\right)) \\
Transformer & 24 & 256 & 1024 & 1.039 & 1.130 & (O\left(T^{2} d\right)) & (O\left(T^{2}+T d\right)) \\
\hline Reformer & 12 & 512 & 1024 & 1.040 & 1.195 & (O(T \log T d)) & (O(T \log T+T d)) \\
Synthesizer & 12 & 512 & 1024 & 0.994 & 1.298 & (O\left(T^{2} d\right)) & (O\left(T^{2}+T d\right)) \\
Linear Transformer & 12 & 512 & 1024 & 0.981 & 1.207 & (O\left(T d^{2}\right)) & (O\left(T d+d^{2}\right)) \\
Performer & 12 & 512 & 1024 & 1.002 & 1.199 & (O\left(T d^{2} \log d\right)) & (O\left(T d \log d+d^{2} \log d\right)) \\
AFT-simple & 12 & 512 & 1024 & 0.854 & 1.180 & (O(T d)) & (O(T d)) \\
\hline RWKV-RNN & 6 & 512 & 1024 & 0.720 & - & (O( T d )) & (O( d )) \\
\hline
\end{tabular}

Table 5: Enwik8 results, measured in bits per character (bpc): the lower the better. Baseline comparisons are made with Reformer (Kitaev et al., 2020), Synthesizer (Tay et al., 2020) (the best performing dense version), Linear Transformer (Katharopoulos et al., 2020), Performer (Choromanski et al., 2020). (L, d), and (T) denote the number of blocks (network depth), dimension of features, and sequence length, respectively. Both Linear Transformer and Performer are implemented with customized CUDA kernels (github.com/idiap/fast-transformers), and all other models are implemented in native Pytorch.

quences. For GPU experiments, we use an NVIDIA A100 with 80GB of VRAM. For CPU experiments, we use an AMD EPYC processor with (30 CPU) cores and (200 GiB) RAM.

Figure 10: Text generation inference memory (CPU RAM, GPU VRAM) for LLMs. Model parameters are not accounted.

Figure 11: Text generation inference time for LLMs. 

\begin{tabular}{llrrrrr}
\hline Task Name & (\begin{array}{l}\text { Measure } \\
\text { type }\end{array}) & (\begin{array}{r}\text { ChatGPT } \\
\text { [%] }\end{array}) & (\begin{array}{r}\text { GPT-4 } \\
\text { [%] }\end{array}) & (\begin{array}{r}\text { RWKV-4 } \\
\text { GPT [%] }\end{array}) & (\begin{array}{r}\text { RWKV-4 } \\
\text { changed [%] }\end{array}) & (\begin{array}{r}\text { SOTA } \\
{[%]}\end{array}) \\
\hline RTE & F1 Macro & 88.1 & (9 1 . 3) & 44.2 & 74.8 & 92.1 \\
WNLI & Accuracy & 81.7 & (9 1 . 6) & 47.9 & 49.3 & 97.9 \\
GoEmotions & F1 Macro & (2 5 . 6) & 23.1 & 7.9 & 7.9 & 52.8 \\
PolEmo2 & F1 Macro & (4 4 . 1) & 41.0 & 38.2 & 40.9 & 76.4 \\
\hline
\end{tabular}

Table 6: ChatGPT, GPT-4 and RWKV-4-Raven-14B reasoning performance comparison in RTE (Wang et al., 2019), WNLI (Wang et al., 2018), GoEmotions (Demszky et al., 2020), and PolEmo2 (Kocoń et al., 2019) benchmarks. SOTA is provided as a supplementary reference.

\section{J Importance of prompt construction and comparison to GPT models}

Inspired by article (Kocoń et al., 2023), we compared the zero-shot performance of the RWKV4-Raven-14B with ChatGPT (access in February 2023) and GPT-4 using several known NLP tasks, i.e., recognizing textual entailment (RTE), Winograd Natural Language Inference (WNLI), and recognizing emotions elicited in readers (GoEmotions and PolEmo2). Each model got the same prompts manually chosen to receive proper responses from the ChatGPT model. As shown in Tab. 6, RWKV performs significantly worse than ChatGPT and GPT-4 in specific task performance. We suspect that this disparity is likely caused by the choice of prompts used to generate the answers. Given that prompts are in natural language and do not consider that RWKV is an RNN, so it can not look back inside an instruction.

When the instruction style was adapted to respect that RNNs is not capable for retrospective processing, quality on some datasets increased significantly (ex. for RTE (Wang et al., 2019) F1 Macro increased from (44.2 %) to (74.8 %) ). We hypothesize that RWKV models are more sensitive to the position of the components in the context, as RNN-based architectures cannot look back and readjust the weight of previous information. For better performance, desired information should be after the question. Example of ChatGPT prompt to RTE:

Having premise <here is a premise (>) judge if the following hypothesis <here is a hypothesis (>) are logically connected with the premise? Answer "entailment" if yes, or "not_entailment" if no.

RWKV prompt taking into account the characteristics of the RNN:

Can you tell me if the hypothesis is entailment or is not entailment to the premise?

\begin{tabular}{llrrr}
\hline Task Name & (\begin{array}{l}\text { Measure } \\
\text { type }\end{array}) & (\begin{array}{r}\text { ChatGPT } \\
{[%]}\end{array}) & (\begin{array}{r}\text { RWKV-4 } \\
\text { adapted [%] }\end{array}) & (\begin{array}{r}\text { SOTA } \\
{[%]}\end{array}) \\
\hline Aggression & F1 Macro & (6 9 . 1 0) & 56.66 & 74.45 \\
MathQA & Accuracy & 71.40 & (8 0 . 6 9) & 83.20 \\
Sarcasm & F1 Macro & 49.88 & (5 0 . 9 6) & 53.57 \\
TweetSent & F1 Macro & (6 3 . 3 2) & 52.50 & 72.07 \\
Unhealthy & F1 Macro & (4 5 . 2 1) & 43.30 & 50.96 \\
\hline
\end{tabular}

Table 7: ChatGPT and RWKV-4-Raven-14B performance comparison in Aggresion (Wulczyn et al., 2017), Sarcasm (Siddiqui, 2019), Unhealthy (Price et al., 2020), MathQA (Cobbe et al., 2021), and TweetSent (Barbieri et al., 2020) benchmarks. SOTA is provided as a supplementary reference.

premise: <here is a premise (>) hypothesis: <here is a hypothesis>

While separating the instruction from the input is relatively easy to do, other aspects of prompt engineering are harder to quantify. Testing the approach of stating the input after the question on multiple other tasks, shown in tab. 7, suggests that better prompts might reduce the disparity between models. Raven achieves comparable result to ChatGPT on unhealthy conversation detection and even surpasses it on the sarcasm detection dataset. While this approach in prompting looks necessary, it alone is not enough to replace the capability of having free access to the whole context. Therefore, prompt engineering seems to be of significantly more importance to the RNN models compared to standard transformers. It is entirely possible that good prompts to (RNN) models do not mean additional restrictions, but should simply be constructed using completely different guidelines. While authors of a forementioned paper (Kocoń et al., 2023) perform chain-of-thought to improve results on MathQA dataset, trying the same approach on the Raven model achieved a very low accuracy of (48.11 %). Despite that, using a very similar prompt without the chain-of-thought outperformed ChatGPT on this dataset. It would not be surprising if, thanks to the hidden state, RNN models do not need additional steps during inference, and adding those only makes the model forget the information it analyzed and deemed important. This further emphasizes the importance of the order of information the model receives.

The template used to prompt the Raven model in MathQA with chain-of-thought prompting:

Write the reasoning and highlight the answer to the question at the end in format: 'Answer: '. The question is: <here is question> The template used to prompt the Raven model in MathQA without chain-of-thought prompting:

Write the answer to the math question in the format:

'Answer: '.

The question is: <here is question> 

\section{K Cases}

In this part, we present a few instances of outputs produced by RWKV model using a Chat interface ({ }^{56}).

\section{RWKV-4-Raven-14B}

\section{Tell me about ravens.}

Ravens are large, black birds with a distinctive white head and neck. They are found in most parts of the world, including North America, Europe, Asia, and Australia. Ravens are known for their intelligence and problem-solving abilities. They are also considered to be symbols of death and transformation in many cultures. Ravens are often associated with the afterlife or death because they have been known to eat carrion or even other birds. In some cultures, ravens are also believed to be messengers of the gods or guardians of treasure.

\section{RWKV-4-Raven-14B}

\section{Explain the concept of machine learning to a 10-year-old.}

Machine learning is like a super-smart computer that can learn on its own. It can recognize patterns and make decisions based on what it has learned. Just like how you might remember something from the past, the computer can remember what it's learned and use it to make predictions or decisions in the future. It's like having a smart friend who always knows what to do!

\section{RWKV-4-Raven-14B}

\section{Write a short poem describing nature's beauty in springtime.}

Nature's beauty in springtime is rare, A symphony of colors, a gentle surprise. The flowers bloom, the birds sing and sing, As the sun shines down, warming our skin.

The trees sway in the gentle breeze, As we take in their beauty with ease. The sky is blue, the clouds are white, A sight to behold, it's truly divine.

({ }^{5} https : / /) github.com/BlinkDL/ChatRWKV

({ }^{6}) https://huggingface.co/spaces/BlinkDL/ChatRWKV-gradio RWKV-4-Raven-14B

Below is the numerical stable formula for updating the state variable in RWKV:

\[
\begin{aligned}
a_{t}^{*} & =e^{p-q} a_{t-1}+e^{u+k_{t}-q} v_{t} \\
b_{t}^{*} & =e^{p-q} b_{t-1}+e^{u+k_{t}-q} \\
w k v_{t} & =\frac{a_{t}^{*}}{b_{t}^{*}}
\end{aligned}
\]

Please write out the corresponding pytorch code to do so, where both (a_{t}) and (b_{t}) are torch.Tensor (([n]))

Here is the PyTorch code to update the state variable in RWKV using the given formula:


This code defines a function update_state that takes in the state variables a and (b), the control variables (u) and (v), and the parameters wkv and torch.randn. It then uses the PyTorch torch.exp function to compute the exponential of the control variables and the log of the normalization factor. Finally, it uses these values to update the state.

\section{RWKV-4-Raven-14B}

\section{Explain the following metaphor: Life is like cats.}

The metaphor "Life is like cats" suggests that life can be unpredictable and full of unexpected twists and turns, just like a cat's behavior. It implies that we cannot fully control or predict the outcome of our lives, just as we cannot control the movements of a cat. The metaphor also implies that life is complex and multifaceted, with many different variables and factors at play. Just as a cat can have multiple personalities or moods, life can have many twists and turns that are difficult to predict or understand. Overall, the metaphor suggests that life is complex and unpredictable, much like a cat's behavior. RWKV-4-Raven-14B

Test the convergence of (\sum_{i=0}^{\infty} 2^{-i}).

The given series is a geometric series with the first term (a_{0}=2) and common ratio (r=\frac{1}{2}). We can find the sum of this series using the formula for a geometric series:

\[
\sum_{i=0}^{\infty} a_{i}=\frac{a_{0}}{1-r}=\frac{\frac{1}{2}}{1-\frac{1}{2}}=\frac{1}{1-\frac{1}{2}}=\frac{2}{1}=2
\]

So, the sum of the given series is 2 .

\section{RWKV-4-Raven-14B}

I would like to cook some Chinese food at home. Do you have any suggestions on the meal I could choose?

Yes, of course! If you are not very experienced in cooking Chinese food, you could start with making noodles or rice, then make a dish using one of those. I also think making stir-fry with some meat is an excellent idea.

I want to make something more special. What about red braised pork belly?

Yes, red braised pork belly is also very popular and tasty.

Tell me the steps to cook.

You will need a few ingredients such as pork belly, star anise, sugar, ginger, scallions and wine. Put all of these in a pot, cook on medium heat and simmer until the meat is tender. After that, remove the meat from the pot and reserve the sauce. Slice the meat and add some ginger slices into it, then boil until it's almost done.

\section{What do I need to be careful with the most during the cooking process?}

To cook pork belly properly, it is very important to monitor the heat of the stove and be patient with the cooking process. Overcooking it can cause the pork to be tough, dry and difficult to chew. You can add a few ingredients like soy sauce or black beans to give the pork more flavor.

Tell me more about the proper heat and timing.

If the temperature of the water is too high, it can damage the collagen and connective tissues, resulting in tough pork belly. It is essential to control the temperature carefully, especially when braising the pork. Additionally, keep the water simmering but not boiling to maintain a low simmer, as high heat will dry out the pork belly and result in tough and unpalatable meat. Lastly, monitor the heat during cooking and make sure the heat is medium, as this is what is most appropriate for slow cooking pork belly.

\section{I see. Thank you for your advice.}

You're welcome! It was a pleasure chatting with you. Let me know if you need any further assistance in the future. RWKV-4-Raven-14B

If all apples are fruits and all fruits are sweet, are all apples sweet? Please explain your answer in detail.

Yes, all apples are fruits, and all fruits are sweet. Therefore, it is safe to assume that all apples are sweet.

If birds can fly. Penguins are birds. Can penguins fly? Please explain your answer in detail.

No, penguins cannot fly. Penguins are birds, but they are not capable of flight like other birds. They have wings and can glide for short distances, but they cannot take off or fly long distances like other birds.

The day before two days after the day before tomorrow is Saturday. What day is it today? Please explain your answer in detail.

Today is Friday. Two days after tomorrow is Sunday. Tomorrow is Saturday.

If five cats can catch five mice in five minutes, how long will it take one cat to catch one mouse? Please explain your answer in detail.

If five cats can catch five mice in five minutes, it means that each cat can catch one mouse in one minute. To find out how long it will take one cat to catch one mouse, we can use the concept of proportionality.

Let's assume that it takes (x) minutes for a cat to catch a mouse. Then, the number of mice caught by a cat in one minute is equal to the number of cats multiplied by the number of mice caught by each cat: (x=5 * 1=5)

So, it will take 5 minutes for one cat to catch one mouse.

\section{RWKV-4-Raven-14B}

Summarize the following paragraph.

<The Abstract of this paper goes here>

The paper presents a novel architecture, Receptance Weighted Key Value (RWKV), that combines the efficient parallelizable training of Transformers with the efficient inference of RNNs. The model is able to scale to tens of billions of parameters and exhibits linear computational complexity during training and inference, making it a promising alternative to Transformers for sequence processing tasks.

\section{Please further summarize it in one sentence.}

The researchers have developed a new architecture that combines the efficient parallelizable training of Transformers with the efficient inference of RNNs, which allows for linear scaling in memory and computational requirements.

In [54]:
help(display_markdown)

Help on function display_markdown in module IPython.core.display:

display_markdown(*objs, **kwargs)
    Displays the Markdown representation of an object.
    
    Parameters
    ----------
    *objs : object
        The Python objects to display, or if raw=True raw markdown data to
        display.
    raw : bool
        Are the data objects raw data or Python objects that need to be
        formatted before display? [default: False]
    metadata : dict (optional)
        Metadata to be associated with the specific mimetype output.

